In [2]:
import requests
import json

url = "https://explorer.natureserve.org/api/natureserve/v1/species/search"

payload = { #
    "locationCriteria": [
        {
            "paramType": "subnation",
            "subnation": "US-SC"
        }
    ],
    "statusCriteria": [
        {
            "paramType": "globalRank",
            "globalRank": "G1"
        },
        {
            "paramType": "globalRank",
            "globalRank": "G2"
        }
    ],
    "pagingOptions": {
        "page": 0,
        "recordsPerPage": 100
    },
    "classificationOptions": {
        "includeInfraspecies": False
    },
    "speciesTaxonomyCriteria": [],
    "textCriteria": [],
    "recordSubtypeCriteria": [],
    "modifiedSince": None
}

headers = {
    "Content-Type": "application/json"
}

try:
    response = requests.post(url, headers=headers, data=json.dumps(payload))
    print(f"Status code: {response.status_code}")

    if response.status_code != 200:
        print("Request failed:", response.text)
    else:
        data = response.json()
        results = data.get("results", [])
        print(f"Returned {len(results)} species.\n")

        for species in results:
            name = species.get("primaryCommonName", "No Common Name")
            sci = species.get("scientificName", "Unknown")
            rank = species.get("globalRank", "No Rank")
            print(f"{name} ({sci}) - {rank}")

except Exception as e:
    print("Error occurred:", str(e))


Status code: 404
Request failed: {"timestamp":"2025-06-10T12:32:15.109+00:00","status":404,"error":"Not Found","path":"/api/natureserve/v1/species/search"}


In [4]:
import requests

url = (
    "https://explorer.natureserve.org/explorer-maps/species_subnational_ranks/"
    "ELEMENT_GLOBAL.2.102187/FeatureServer/0/query"
    "?where=subnationalCode%3D%27US-SC%27%20AND%20(rank%20IN%20(%27S1%27,%27S2%27))"
    "&outFields=subnationalCode,subnationalName,rank"
    "&f=geojson"
)

resp = requests.get(url)
print(resp.status_code)
data = resp.json()

print(f"Number of features: {len(data.get('features', []))}")
for feature in data.get("features", []):
    props = feature.get("properties", {})
    print(f"{props.get('subnationalName')}: {props.get('rank')}")


200
Number of features: 0


In [5]:
import requests

url = (
    "https://services.arcgis.com/f8bcmNGBV6fYyrhC/arcgis/rest/services/US_Subnational_Ranks/FeatureServer/0/query"
    "?where=subnationalCode%3D%27US-SC%27%20AND%20(rank%20IN%20(%27S1%27,%27S2%27))"
    "&outFields=primaryCommonName,scientificName,subnationalCode,rank"
    "&f=geojson"
)

resp = requests.get(url)
print(resp.status_code)
data = resp.json()

features = data.get("features", [])
print(f"Returned {len(features)} features.\n")
for f in features[:10]:  # show just first 10
    p = f["properties"]
    print(f"{p.get('primaryCommonName')} ({p.get('scientificName')}) - {p.get('rank')}")


400


JSONDecodeError: Expecting value: line 1 column 1 (char 0)

In [6]:
import requests

# Example: Get species data for California
url = "https://explorer.natureserve.org/api/data/speciesSearch"
params = {
    "stateProvince": "CA",
    "nation": "US",
    "format": "json"
}

response = requests.get(url, params=params)
data = response.json()

JSONDecodeError: Expecting value: line 1 column 1 (char 0)

In [7]:
import requests
import json

def get_endangered_species_sc():
    """Get endangered species in South Carolina with geometries"""

    base_url = "https://explorer.natureserve.org/api/data/"

    # First, search for endangered species in SC
    search_params = {
        'locationName': 'South Carolina',
        'globalRank': 'G1,G2',  # Critically imperiled to imperiled
        'stateRank': 'S1,S2',   # State level imperiled
        'conservationStatus': 'federally_listed',
        'format': 'json'
    }

    try:
        # Species search
        response = requests.get(
            f"{base_url}speciesSearch",
            params=search_params,
            timeout=30
        )

        if response.status_code == 200:
            species_data = response.json()
            print(f"Found {len(species_data.get('results', []))} species")

            # Get detailed info with geometries for each species
            detailed_results = []

            for species in species_data.get('results', [])[:10]:  # Limit to first 10
                species_id = species.get('elementGlobalId') or species.get('id')

                if species_id:
                    # Get detailed species info including range/geometry
                    detail_response = requests.get(
                        f"{base_url}species/{species_id}",
                        params={'includeGeometry': 'true', 'format': 'json'},
                        timeout=30
                    )

                    if detail_response.status_code == 200:
                        detailed_data = detail_response.json()
                        detailed_results.append({
                            'name': species.get('scientificName'),
                            'common_name': species.get('commonName'),
                            'global_rank': species.get('globalRank'),
                            'state_rank': species.get('stateRank'),
                            'geometry': detailed_data.get('geometry') or detailed_data.get('range'),
                            'full_data': detailed_data
                        })

                        print(f"Got geometry for: {species.get('scientificName')}")

            return detailed_results

        else:
            print(f"Search failed: {response.status_code}")
            return None

    except requests.exceptions.RequestException as e:
        print(f"Request error: {e}")
        return None

# Alternative approach using occurrence data
def get_species_occurrences_sc():
    """Get species occurrence points in South Carolina"""

    params = {
        'state': 'SC',
        'conservationStatus': 'endangered',
        'includeGeometry': 'true',
        'format': 'geojson'  # Request GeoJSON format directly
    }

    try:
        response = requests.get(
            "https://explorer.natureserve.org/api/data/occurrences",
            params=params,
            timeout=30
        )

        if response.status_code == 200:
            return response.json()
        else:
            print(f"Occurrence search failed: {response.status_code}")
            return None

    except requests.exceptions.RequestException as e:
        print(f"Request error: {e}")
        return None

# Run the queries
if __name__ == "__main__":
    print("Searching for endangered species in South Carolina...")

    # Try species search first
    species_results = get_endangered_species_sc()

    if species_results:
        print(f"\nFound {len(species_results)} species with geometry data")

        # Save results
        with open('sc_endangered_species.json', 'w') as f:
            json.dump(species_results, f, indent=2)

    # Also try occurrence data
    print("\nTrying occurrence data...")
    occurrence_data = get_species_occurrences_sc()

    if occurrence_data:
        with open('sc_species_occurrences.geojson', 'w') as f:
            json.dump(occurrence_data, f, indent=2)
        print("Saved occurrence data as GeoJSON")

Searching for endangered species in South Carolina...
Search failed: 405

Trying occurrence data...
Occurrence search failed: 404


In [8]:
import requests
import json

def explore_natureserve_api():
    """Try to find working NatureServe API endpoints"""

    # Common base URLs to try
    base_urls = [
        "https://explorer.natureserve.org/api/",
        "https://services.natureserve.org/",
        "https://api.natureserve.org/",
        "https://explorer.natureserve.org/api/data/",
    ]

    # Common endpoint patterns
    endpoints = [
        "",  # Root to see API info
        "species",
        "search",
        "taxa",
        "conservation",
        "explorer",
        "v1/",
        "rest/",
    ]

    for base_url in base_urls:
        print(f"\nTrying base URL: {base_url}")

        for endpoint in endpoints:
            try:
                url = f"{base_url}{endpoint}"
                response = requests.get(url, timeout=10)

                if response.status_code == 200:
                    print(f"✓ Working: {url}")
                    print(f"  Response type: {response.headers.get('content-type', 'unknown')}")

                    # Try to parse response
                    try:
                        if 'json' in response.headers.get('content-type', ''):
                            data = response.json()
                            print(f"  Keys: {list(data.keys()) if isinstance(data, dict) else 'Not a dict'}")
                    except:
                        print(f"  Text preview: {response.text[:200]}...")

                elif response.status_code == 405:
                    print(f"? Method not allowed: {url} (might need POST)")
                elif response.status_code == 404:
                    print(f"✗ Not found: {url}")
                else:
                    print(f"? Status {response.status_code}: {url}")

            except requests.exceptions.RequestException as e:
                print(f"✗ Error: {url} - {e}")

# Alternative: Try the NatureServe Explorer web interface approach
def try_explorer_search():
    """Try to mimic what the web interface does"""

    # This might be closer to what the web interface uses
    urls_to_try = [
        "https://explorer.natureserve.org/Taxon/ELEMENT_GLOBAL.2.154701/Etheostoma_collis_carolinae",
        "https://explorer.natureserve.org/api/search?q=South%20Carolina",
        "https://explorer.natureserve.org/api/taxa/search?location=South%20Carolina",
    ]

    for url in urls_to_try:
        try:
            response = requests.get(url, timeout=10)
            print(f"{url}: {response.status_code}")

            if response.status_code == 200:
                print(f"Content-Type: {response.headers.get('content-type')}")
                print(f"Preview: {response.text[:300]}...")

        except Exception as e:
            print(f"Error with {url}: {e}")

if __name__ == "__main__":
    print("Exploring NatureServe API endpoints...")
    explore_natureserve_api()

    print("\n" + "="*50)
    print("Trying Explorer-specific URLs...")
    try_explorer_search()

Exploring NatureServe API endpoints...

Trying base URL: https://explorer.natureserve.org/api/
✗ Not found: https://explorer.natureserve.org/api/
✗ Not found: https://explorer.natureserve.org/api/species
✗ Not found: https://explorer.natureserve.org/api/search
✗ Not found: https://explorer.natureserve.org/api/taxa
✗ Not found: https://explorer.natureserve.org/api/conservation
✗ Not found: https://explorer.natureserve.org/api/explorer
✗ Not found: https://explorer.natureserve.org/api/v1/
✗ Not found: https://explorer.natureserve.org/api/rest/

Trying base URL: https://services.natureserve.org/
✓ Working: https://services.natureserve.org/
  Response type: text/html;charset=UTF-8
✓ Working: https://services.natureserve.org/species
  Response type: text/html;charset=UTF-8
✓ Working: https://services.natureserve.org/search
  Response type: text/html;charset=UTF-8
✓ Working: https://services.natureserve.org/taxa
  Response type: text/html;charset=UTF-8
✓ Working: https://services.natureserve

In [9]:
import requests
import json

def try_services_natureserve():
    """Try the services.natureserve.org endpoints"""

    # These returned HTML, so they might have API documentation or forms
    base_url = "https://services.natureserve.org/"

    # Try to find API documentation or JSON endpoints
    endpoints_to_try = [
        "api/",
        "api/v1/",
        "api/species/",
        "api/search/",
        "rest/api/",
        "rest/v1/",
        "imap/services/",
        "ecoregions/",
        "biodiversity/"
    ]

    headers = {
        'Accept': 'application/json',
        'User-Agent': 'Mozilla/5.0 (compatible; research bot)'
    }

    for endpoint in endpoints_to_try:
        try:
            url = f"{base_url}{endpoint}"
            response = requests.get(url, headers=headers, timeout=10)

            print(f"{url}: {response.status_code}")

            if response.status_code == 200:
                content_type = response.headers.get('content-type', '')
                if 'json' in content_type:
                    print(f"  JSON response found!")
                    try:
                        data = response.json()
                        print(f"  Keys: {list(data.keys()) if isinstance(data, dict) else type(data)}")
                    except:
                        pass
                else:
                    # Look for API info in HTML
                    if 'api' in response.text.lower():
                        print(f"  HTML mentions 'api'")

        except Exception as e:
            print(f"  Error: {e}")

def try_post_search():
    """Try the POST endpoint that returned 405 for GET"""

    url = "https://explorer.natureserve.org/api/data/search"

    # Try different POST payloads
    payloads = [
        {
            "location": "South Carolina",
            "conservationStatus": "endangered",
            "includeGeometry": True
        },
        {
            "query": "South Carolina endangered species",
            "format": "json"
        },
        {
            "filters": {
                "location": "South Carolina",
                "globalRank": ["G1", "G2"],
                "stateRank": ["S1", "S2"]
            }
        },
        {
            "searchText": "South Carolina",
            "taxonomicGroup": "all",
            "conservationConcern": True
        }
    ]

    headers = {
        'Content-Type': 'application/json',
        'Accept': 'application/json',
        'User-Agent': 'Mozilla/5.0 (compatible; research bot)'
    }

    for i, payload in enumerate(payloads):
        try:
            print(f"\nTrying POST payload {i+1}:")
            print(f"Payload: {json.dumps(payload, indent=2)}")

            response = requests.post(url, json=payload, headers=headers, timeout=15)

            print(f"Status: {response.status_code}")

            if response.status_code == 200:
                print("SUCCESS! Response:")
                try:
                    data = response.json()
                    print(json.dumps(data, indent=2)[:500] + "...")
                except:
                    print(response.text[:500] + "...")
            elif response.status_code == 400:
                print("Bad request - check payload format")
                print(response.text[:200])
            else:
                print(f"Response: {response.text[:200]}")

        except Exception as e:
            print(f"Error: {e}")

def try_other_endpoints():
    """Try some other common patterns"""

    # Based on the working taxon URL, try similar patterns
    base_urls = [
        "https://explorer.natureserve.org/",
        "https://services.natureserve.org/"
    ]

    endpoints = [
        "api/taxon/search",
        "api/elements/search",
        "api/location/search",
        "rest/taxon",
        "rest/elements",
        "imap/rest/element/search"
    ]

    for base in base_urls:
        for endpoint in endpoints:
            try:
                url = f"{base}{endpoint}"

                # Try GET first
                response = requests.get(url, timeout=10)
                if response.status_code == 200:
                    print(f"GET {url}: SUCCESS")
                elif response.status_code == 405:
                    # Try POST
                    response = requests.post(url, json={"query": "test"}, timeout=10)
                    if response.status_code != 404:
                        print(f"POST {url}: {response.status_code}")

            except:
                pass

if __name__ == "__main__":
    print("=== Trying services.natureserve.org ===")
    try_services_natureserve()

    print("\n=== Trying POST to search endpoint ===")
    try_post_search()

    print("\n=== Trying other endpoint patterns ===")
    try_other_endpoints()

=== Trying services.natureserve.org ===
https://services.natureserve.org/api/: 200
  HTML mentions 'api'
https://services.natureserve.org/api/v1/: 200
  HTML mentions 'api'
https://services.natureserve.org/api/species/: 200
  HTML mentions 'api'
https://services.natureserve.org/api/search/: 200
  HTML mentions 'api'
https://services.natureserve.org/rest/api/: 200
  HTML mentions 'api'
https://services.natureserve.org/rest/v1/: 200
  HTML mentions 'api'
https://services.natureserve.org/imap/services/: 200
  HTML mentions 'api'
https://services.natureserve.org/ecoregions/: 200
  HTML mentions 'api'
https://services.natureserve.org/biodiversity/: 200
  HTML mentions 'api'

=== Trying POST to search endpoint ===

Trying POST payload 1:
Payload: {
  "location": "South Carolina",
  "conservationStatus": "endangered",
  "includeGeometry": true
}
Status: 400
Bad request - check payload format
{"timestamp":"2025-06-10T12:41:33.132+00:00","status":400,"error":"Bad Request","message":"JSON parse 

In [10]:
import requests
import json

def explore_working_endpoints():
    """Check the working endpoints we found"""

    working_endpoints = [
        "https://services.natureserve.org/api/taxon/search",
        "https://services.natureserve.org/api/elements/search",
        "https://services.natureserve.org/api/location/search",
        "https://services.natureserve.org/rest/taxon",
        "https://services.natureserve.org/rest/elements",
        "https://services.natureserve.org/imap/rest/element/search"
    ]

    headers = {
        'Accept': 'application/json',
        'Content-Type': 'application/json',
        'User-Agent': 'Mozilla/5.0 (compatible; research bot)'
    }

    for url in working_endpoints:
        print(f"\n=== Testing: {url} ===")

        try:
            # Try GET first
            response = requests.get(url, headers=headers, timeout=10)
            print(f"GET Status: {response.status_code}")

            if response.status_code == 200:
                try:
                    data = response.json()
                    print(f"JSON Response: {json.dumps(data, indent=2)[:300]}...")
                except:
                    print(f"HTML/Text response: {response.text[:200]}...")

            # Try with query parameters
            params = {
                'location': 'South Carolina',
                'q': 'South Carolina',
                'state': 'SC',
                'region': 'South Carolina'
            }

            for param_name, param_value in params.items():
                try:
                    response = requests.get(url, params={param_name: param_value}, headers=headers, timeout=10)
                    if response.status_code == 200:
                        print(f"GET with {param_name}={param_value}: SUCCESS")
                        try:
                            data = response.json()
                            if data and len(str(data)) > 20:  # Non-empty response
                                print(f"  Response preview: {json.dumps(data, indent=2)[:200]}...")
                                break
                        except:
                            pass
                except:
                    pass

        except Exception as e:
            print(f"Error: {e}")

def try_correct_search_format():
    """Try the POST endpoint with the correct class structure"""

    url = "https://explorer.natureserve.org/api/data/search"

    # Based on the error, try formats that match their class structure
    payloads = [
        {
            "@class": "org.natureserve.nsx.search.criteria.CommonSearchCriteria",
            "location": "South Carolina",
            "conservationStatus": "endangered"
        },
        {
            "criteria": {
                "@type": "CommonSearchCriteria",
                "location": "South Carolina",
                "globalRank": ["G1", "G2"]
            }
        },
        {
            "searchCriteria": {
                "location": "South Carolina",
                "taxonomicGroup": "all",
                "conservationConcern": True
            }
        },
        # Try simpler format
        {
            "locationName": "South Carolina",
            "globalRank": "G1,G2",
            "stateRank": "S1,S2"
        }
    ]

    headers = {
        'Content-Type': 'application/json',
        'Accept': 'application/json'
    }

    for i, payload in enumerate(payloads):
        try:
            print(f"\nTrying corrected payload {i+1}:")
            response = requests.post(url, json=payload, headers=headers, timeout=15)
            print(f"Status: {response.status_code}")

            if response.status_code == 200:
                print("SUCCESS!")
                try:
                    data = response.json()
                    print(json.dumps(data, indent=2)[:500])
                except:
                    print(response.text[:300])
            else:
                print(f"Response: {response.text[:200]}")

        except Exception as e:
            print(f"Error: {e}")

def try_imap_service():
    """Try the imap service which might be their mapping/geometry service"""

    base_url = "https://services.natureserve.org/imap/"

    endpoints = [
        "rest/element/search",
        "rest/species/search",
        "services/element/search",
        "services/species/search"
    ]

    params_to_try = [
        {'state': 'SC', 'endangered': 'true'},
        {'location': 'South Carolina'},
        {'q': 'endangered species South Carolina'},
        {'region': 'South Carolina', 'conservation': 'G1,G2'}
    ]

    headers = {'Accept': 'application/json'}

    for endpoint in endpoints:
        url = f"{base_url}{endpoint}"
        print(f"\n=== Testing iMap: {url} ===")

        for params in params_to_try:
            try:
                response = requests.get(url, params=params, headers=headers, timeout=10)
                if response.status_code == 200:
                    print(f"Success with params: {params}")
                    try:
                        data = response.json()
                        if data:
                            print(f"Response: {json.dumps(data, indent=2)[:300]}...")
                            return  # Found working endpoint
                    except:
                        if len(response.text) > 50:
                            print(f"Text response: {response.text[:200]}...")
            except:
                pass

if __name__ == "__main__":
    print("=== Exploring working endpoints ===")
    explore_working_endpoints()

    print("\n=== Trying corrected POST format ===")
    try_correct_search_format()

    print("\n=== Trying iMap service ===")
    try_imap_service()

=== Exploring working endpoints ===

=== Testing: https://services.natureserve.org/api/taxon/search ===
GET Status: 200
HTML/Text response: <!DOCTYPE html>
<html lang="en">
<head>
<meta charset="UTF-8">
<meta http-equiv="X-UA-Compatible" content="IE=edge">
<meta name="viewport" content="width=device-width, initial-scale=1.0">
<meta ...
GET with location=South Carolina: SUCCESS
GET with q=South Carolina: SUCCESS
GET with state=SC: SUCCESS
GET with region=South Carolina: SUCCESS

=== Testing: https://services.natureserve.org/api/elements/search ===
GET Status: 200
HTML/Text response: <!DOCTYPE html>
<html lang="en">
<head>
<meta charset="UTF-8">
<meta http-equiv="X-UA-Compatible" content="IE=edge">
<meta name="viewport" content="width=device-width, initial-scale=1.0">
<meta ...
GET with location=South Carolina: SUCCESS
GET with q=South Carolina: SUCCESS
GET with state=SC: SUCCESS
GET with region=South Carolina: SUCCESS

=== Testing: https://services.natureserve.org/api/location/search ==

In [11]:
import requests
import json
import re

def inspect_html_for_api_calls():
    """Look at the HTML to find JavaScript API calls"""

    # Get the main page HTML and look for API endpoints
    urls_to_inspect = [
        "https://explorer.natureserve.org/",
        "https://services.natureserve.org/",
        "https://explorer.natureserve.org/Taxon/ELEMENT_GLOBAL.2.154701/Etheostoma_collis_carolinae"
    ]

    for url in urls_to_inspect:
        try:
            response = requests.get(url, timeout=10)
            if response.status_code == 200:
                html = response.text

                print(f"\n=== Analyzing {url} ===")

                # Look for API endpoints in JavaScript
                api_patterns = [
                    r'api[/"\s]*[:\s]*["\']([^"\']+)["\']',
                    r'endpoint[/"\s]*[:\s]*["\']([^"\']+)["\']',
                    r'url[/"\s]*[:\s]*["\']([^"\']*api[^"\']*)["\']',
                    r'fetch\(["\']([^"\']*api[^"\']*)["\']',
                    r'ajax[^{]*url[^"\']*["\']([^"\']*api[^"\']*)["\']',
                    r'/api/[a-zA-Z0-9/._-]+',
                    r'https://[^"\'\s]*api[^"\'\s]*'
                ]

                found_apis = set()
                for pattern in api_patterns:
                    matches = re.findall(pattern, html, re.IGNORECASE)
                    found_apis.update(matches)

                if found_apis:
                    print("Found potential API endpoints:")
                    for api in sorted(found_apis):
                        if len(api) > 5:  # Filter out very short matches
                            print(f"  {api}")

                # Look for specific NatureServe API patterns
                if 'explorer.natureserve.org' in html:
                    explorer_matches = re.findall(r'explorer\.natureserve\.org[^"\'\s]*', html)
                    if explorer_matches:
                        print("Explorer URLs found:")
                        for match in set(explorer_matches):
                            print(f"  {match}")

        except Exception as e:
            print(f"Error inspecting {url}: {e}")

def try_alternative_api_formats():
    """Try different API endpoint formats based on common patterns"""

    # Since we know the explorer has a working taxon page, try to reverse engineer
    base_urls = [
        "https://explorer.natureserve.org/api/",
        "https://explorer.natureserve.org/",
        "https://services.natureserve.org/api/"
    ]

    # Common REST API patterns
    endpoints = [
        "v1/taxa",
        "v1/species",
        "v1/search",
        "v1/elements",
        "taxa/search",
        "species/search",
        "elements/search",
        "search/taxa",
        "search/species",
        "search/elements"
    ]

    headers = {
        'Accept': 'application/json, text/plain, */*',
        'Content-Type': 'application/json',
        'User-Agent': 'Mozilla/5.0 (Macintosh; Intel Mac OS X 10_15_7) AppleWebKit/537.36'
    }

    for base in base_urls:
        for endpoint in endpoints:
            url = f"{base}{endpoint}"

            try:
                # Try GET
                response = requests.get(url, headers=headers, timeout=5)
                if response.status_code == 200 and 'json' in response.headers.get('content-type', ''):
                    print(f"Found JSON API: GET {url}")
                    try:
                        data = response.json()
                        print(f"Sample: {json.dumps(data, indent=2)[:200]}...")
                    except:
                        pass

                # Try GET with query params
                for param in ['q=endangered', 'location=South+Carolina', 'state=SC']:
                    try:
                        response = requests.get(f"{url}?{param}", headers=headers, timeout=5)
                        if response.status_code == 200 and 'json' in response.headers.get('content-type', ''):
                            print(f"Found JSON API: GET {url}?{param}")
                            break
                    except:
                        pass

            except:
                pass

def try_direct_taxon_api():
    """Try to use the taxon URL pattern we know works"""

    # We know this works: https://explorer.natureserve.org/Taxon/ELEMENT_GLOBAL.2.154701/Etheostoma_collis_carolinae
    # Try to find the API equivalent

    api_variants = [
        "https://explorer.natureserve.org/api/taxon/ELEMENT_GLOBAL.2.154701",
        "https://explorer.natureserve.org/api/Taxon/ELEMENT_GLOBAL.2.154701",
        "https://explorer.natureserve.org/api/v1/taxon/ELEMENT_GLOBAL.2.154701",
        "https://explorer.natureserve.org/api/data/taxon/ELEMENT_GLOBAL.2.154701",
        "https://explorer.natureserve.org/taxon/ELEMENT_GLOBAL.2.154701.json",
        "https://explorer.natureserve.org/Taxon/ELEMENT_GLOBAL.2.154701.json"
    ]

    headers = {'Accept': 'application/json'}

    for url in api_variants:
        try:
            response = requests.get(url, headers=headers, timeout=10)
            print(f"{url}: {response.status_code}")

            if response.status_code == 200:
                content_type = response.headers.get('content-type', '')
                if 'json' in content_type:
                    print(f"  SUCCESS! JSON response found")
                    try:
                        data = response.json()
                        print(f"  Sample: {json.dumps(data, indent=2)[:300]}...")
                        return url, data  # Found working API!
                    except:
                        pass
                else:
                    print(f"  HTML response: {response.text[:100]}...")
        except Exception as e:
            pass

    return None, None

if __name__ == "__main__":
    print("=== Inspecting HTML for API calls ===")
    inspect_html_for_api_calls()

    print("\n=== Trying alternative API formats ===")
    try_alternative_api_formats()

    print("\n=== Trying direct taxon API ===")
    api_url, data = try_direct_taxon_api()

    if api_url:
        print(f"\nFound working API: {api_url}")
        print("Now we can build South Carolina queries!")

=== Inspecting HTML for API calls ===

=== Analyzing https://explorer.natureserve.org/ ===
Found potential API endpoints:
  https://fonts.googleapis.com/css?family=Roboto:300,400,500,700&display=swap

=== Analyzing https://services.natureserve.org/ ===
Found potential API endpoints:
  /api/data/cosewic
  /api/data/dataSensitive
  /api/data/dataSensitiveCategory
  /api/data/dataSensitiveCategory/bySubnation
  /api/data/ecosystemHierarchy/
  /api/data/ecosystemHierarchy/ELEMENT_GLOBAL.2.683060
  /api/data/ecosystemHierarchy/doesNotExist
  /api/data/ecosystemsSearch
  /api/data/informalTaxonomy
  /api/data/ivcDescendants
  /api/data/nameCategory
  /api/data/nations
  /api/data/sara
  /api/data/scientificTaxonomy
  /api/data/search
  /api/data/sensitiveTaxa
  /api/data/sensitiveTaxa/byCategory
  /api/data/speciesSearch
  /api/data/subnations/
  /api/data/subnations/US
  /api/data/taxon/
  /api/data/taxon/ELEMENT_GLOBAL.2.154701
  /api/data/taxonSearch
  /api/data/unpublishedTaxa
  /api/dat

In [12]:
import requests
import json

def search_endangered_species_sc():
    """Search for endangered species in South Carolina using the working API"""

    # Base URL we found working
    base_url = "https://explorer.natureserve.org/api/data/"

    headers = {
        'Accept': 'application/json',
        'User-Agent': 'Mozilla/5.0 (compatible; research bot)'
    }

    print("=== Searching for species in South Carolina ===")

    # Try the speciesSearch endpoint we found
    search_url = f"{base_url}speciesSearch"

    # Try different parameter combinations
    search_params = [
        {
            'locationName': 'South Carolina',
            'globalRank': 'G1,G2,G3',  # At-risk ranks
            'format': 'json'
        },
        {
            'subnation': 'US-SC',  # ISO code for South Carolina
            'conservationConcern': 'true'
        },
        {
            'state': 'SC',
            'endangered': 'true'
        },
        {
            'q': 'South Carolina',
            'rank': 'G1,G2'
        }
    ]

    for i, params in enumerate(search_params):
        try:
            print(f"\nTrying search parameters {i+1}: {params}")
            response = requests.get(search_url, params=params, headers=headers, timeout=15)

            print(f"Status: {response.status_code}")

            if response.status_code == 200:
                try:
                    data = response.json()
                    print(f"Success! Found {len(data.get('results', data))} results")

                    # Show sample results
                    results = data.get('results', data if isinstance(data, list) else [])
                    if results:
                        for j, species in enumerate(results[:3]):  # Show first 3
                            print(f"  {j+1}. {species.get('scientificName', 'Unknown')} - {species.get('commonName', 'No common name')}")
                            print(f"     Global Rank: {species.get('globalRank', 'Unknown')}")
                            print(f"     Element ID: {species.get('elementGlobalId', 'Unknown')}")

                        return results  # Return successful results

                except json.JSONDecodeError:
                    print(f"Response is not JSON: {response.text[:200]}")
                except Exception as e:
                    print(f"Error parsing response: {e}")
            else:
                print(f"Error response: {response.text[:200]}")

        except Exception as e:
            print(f"Request error: {e}")

    return None

def get_species_geometry(element_id):
    """Get detailed species info including geometry for a specific species"""

    base_url = "https://explorer.natureserve.org/api/data/"
    taxon_url = f"{base_url}taxon/{element_id}"

    headers = {'Accept': 'application/json'}

    try:
        print(f"\nGetting detailed info for element {element_id}...")
        response = requests.get(taxon_url, headers=headers, timeout=15)

        if response.status_code == 200:
            data = response.json()

            print(f"Species: {data.get('scientificName', 'Unknown')}")
            print(f"Common Name: {data.get('primaryCommonName', 'Unknown')}")
            print(f"Global Rank: {data.get('roundedGlobalRank', 'Unknown')}")

            # Look for geometry/range information
            geometry_fields = [
                'geometry', 'range', 'distribution', 'rangeMap',
                'occurrences', 'locations', 'coordinates'
            ]

            found_geometry = False
            for field in geometry_fields:
                if field in data and data[field]:
                    print(f"Found {field}: {type(data[field])}")
                    if isinstance(data[field], (dict, list)):
                        print(f"  Content preview: {str(data[field])[:200]}...")
                    found_geometry = True

            if not found_geometry:
                print("No geometry fields found in this response")
                print("Available fields:", list(data.keys())[:10])

            return data

    except Exception as e:
        print(f"Error getting species details: {e}")
        return None

def try_subnations_endpoint():
    """Try the subnations endpoint to understand location formatting"""

    base_url = "https://explorer.natureserve.org/api/data/"

    # Try getting subnation info for South Carolina
    endpoints_to_try = [
        f"{base_url}subnations/US-SC",
        f"{base_url}subnations/",
        f"{base_url}nations"
    ]

    headers = {'Accept': 'application/json'}

    for url in endpoints_to_try:
        try:
            print(f"\nTrying: {url}")
            response = requests.get(url, headers=headers, timeout=10)

            if response.status_code == 200:
                data = response.json()
                print(f"Success! Response type: {type(data)}")

                if isinstance(data, list):
                    print(f"Found {len(data)} items")
                    # Look for South Carolina
                    for item in data:
                        if isinstance(item, dict):
                            name = item.get('name', item.get('subnationName', ''))
                            code = item.get('code', item.get('subnationCode', ''))
                            if 'south carolina' in name.lower() or 'sc' in code.upper():
                                print(f"Found SC: {item}")
                elif isinstance(data, dict):
                    print(f"Response keys: {list(data.keys())}")
                    print(f"Sample: {json.dumps(data, indent=2)[:300]}...")

        except Exception as e:
            print(f"Error: {e}")

if __name__ == "__main__":
    # First, understand the location codes
    try_subnations_endpoint()

    # Search for endangered species
    species_results = search_endangered_species_sc()

    if species_results:
        print(f"\n=== Getting geometry for first few species ===")

        # Get detailed info for first few species
        for species in species_results[:3]:
            element_id = species.get('elementGlobalId')
            if element_id:
                species_data = get_species_geometry(element_id)
                print("-" * 50)
    else:
        print("No species found. Let's try the taxonSearch endpoint...")

        # Try taxonSearch as alternative
        taxon_search_url = "https://explorer.natureserve.org/api/data/taxonSearch"
        try:
            response = requests.get(taxon_search_url,
                                  params={'q': 'South Carolina endangered'},
                                  headers={'Accept': 'application/json'},
                                  timeout=15)
            print(f"TaxonSearch status: {response.status_code}")
            if response.status_code == 200:
                print(f"TaxonSearch response: {response.text[:300]}...")
        except Exception as e:
            print(f"TaxonSearch error: {e}")


Trying: https://explorer.natureserve.org/api/data/subnations/US-SC

Trying: https://explorer.natureserve.org/api/data/subnations/

Trying: https://explorer.natureserve.org/api/data/nations
Success! Response type: <class 'list'>
Found 54 items
=== Searching for species in South Carolina ===

Trying search parameters 1: {'locationName': 'South Carolina', 'globalRank': 'G1,G2,G3', 'format': 'json'}
Status: 405
Error response: 

Trying search parameters 2: {'subnation': 'US-SC', 'conservationConcern': 'true'}
Status: 405
Error response: 

Trying search parameters 3: {'state': 'SC', 'endangered': 'true'}
Status: 405
Error response: 

Trying search parameters 4: {'q': 'South Carolina', 'rank': 'G1,G2'}
Status: 405
Error response: 
No species found. Let's try the taxonSearch endpoint...
TaxonSearch status: 500


In [13]:
import requests
import json

def try_post_search_with_correct_format():
    """Try POST requests with different payload structures"""

    search_url = "https://explorer.natureserve.org/api/data/search"

    headers = {
        'Content-Type': 'application/json',
        'Accept': 'application/json',
        'User-Agent': 'Mozilla/5.0 (compatible; research bot)'
    }

    # The error mentioned "Could not resolve subtype of [simple type, class org.natureserve.nsx.search.criteria.Common"
    # This suggests we need to specify the search criteria type

    payloads = [
        # Try with explicit type information
        {
            "criteriaType": "SpeciesSearchCriteria",
            "subnation": "US-SC",
            "globalRank": ["G1", "G2", "G3"]
        },
        # Try with different type name
        {
            "criteriaType": "CommonSearchCriteria",
            "locationName": "South Carolina",
            "conservationConcern": True
        },
        # Try array format
        {
            "criteria": [
                {
                    "type": "location",
                    "value": "US-SC"
                },
                {
                    "type": "rank",
                    "value": "G1,G2"
                }
            ]
        },
        # Try the format from their error - maybe we need @class
        {
            "@class": "org.natureserve.nsx.search.criteria.SpeciesSearchCriteria",
            "subnation": "US-SC",
            "globalRank": ["G1", "G2"]
        }
    ]

    for i, payload in enumerate(payloads):
        try:
            print(f"\n=== Trying POST payload {i+1} ===")
            print(f"Payload: {json.dumps(payload, indent=2)}")

            response = requests.post(search_url, json=payload, headers=headers, timeout=20)
            print(f"Status: {response.status_code}")

            if response.status_code == 200:
                print("SUCCESS!")
                data = response.json()
                print(f"Response: {json.dumps(data, indent=2)[:500]}...")
                return data
            else:
                print(f"Error: {response.text[:300]}")

        except Exception as e:
            print(f"Request error: {e}")

    return None

def try_species_search_post():
    """Try the speciesSearch endpoint with POST"""

    species_search_url = "https://explorer.natureserve.org/api/data/speciesSearch"

    headers = {
        'Content-Type': 'application/json',
        'Accept': 'application/json'
    }

    # Try different POST payloads for species search
    payloads = [
        {
            "subnation": "US-SC",
            "globalRank": "G1,G2,G3"
        },
        {
            "locationCriteria": {
                "subnation": "US-SC"
            },
            "rankCriteria": {
                "globalRank": ["G1", "G2", "G3"]
            }
        },
        {
            "searchCriteria": {
                "location": "US-SC",
                "conservationStatus": "at-risk"
            }
        }
    ]

    for i, payload in enumerate(payloads):
        try:
            print(f"\n=== Trying speciesSearch POST {i+1} ===")
            response = requests.post(species_search_url, json=payload, headers=headers, timeout=15)
            print(f"Status: {response.status_code}")

            if response.status_code == 200:
                print("SUCCESS!")
                data = response.json()
                print(f"Found: {len(data.get('results', []))} species")
                return data
            else:
                print(f"Error: {response.text[:200]}")

        except Exception as e:
            print(f"Error: {e}")

    return None

def explore_working_taxon_for_clues():
    """Examine the working taxon API to understand the data structure"""

    # We know this works - let's see what South Carolina data looks like
    taxon_url = "https://explorer.natureserve.org/api/data/taxon/ELEMENT_GLOBAL.2.154701"

    headers = {'Accept': 'application/json'}

    try:
        response = requests.get(taxon_url, headers=headers, timeout=10)
        if response.status_code == 200:
            data = response.json()

            print("=== Examining taxon structure for location info ===")

            # Look for location/geographic information
            location_fields = [
                'subnationalRanks', 'nationalRanks', 'jurisdictions',
                'distribution', 'range', 'locations', 'states', 'provinces'
            ]

            for field in location_fields:
                if field in data:
                    print(f"\nFound {field}:")
                    field_data = data[field]
                    if isinstance(field_data, list) and field_data:
                        for item in field_data[:3]:  # Show first 3
                            print(f"  {item}")
                    elif isinstance(field_data, dict):
                        print(f"  {json.dumps(field_data, indent=4)[:300]}...")
                    else:
                        print(f"  {field_data}")

            # Look specifically for South Carolina
            sc_mentions = []
            def find_sc_in_data(obj, path=""):
                if isinstance(obj, dict):
                    for key, value in obj.items():
                        new_path = f"{path}.{key}" if path else key
                        if isinstance(value, str) and ('south carolina' in value.lower() or 'sc' in value.lower()):
                            sc_mentions.append(f"{new_path}: {value}")
                        find_sc_in_data(value, new_path)
                elif isinstance(obj, list):
                    for i, item in enumerate(obj):
                        find_sc_in_data(item, f"{path}[{i}]")

            find_sc_in_data(data)

            if sc_mentions:
                print(f"\nFound South Carolina references:")
                for mention in sc_mentions:
                    print(f"  {mention}")

            return data

    except Exception as e:
        print(f"Error examining taxon: {e}")
        return None

if __name__ == "__main__":
    print("=== Trying POST requests to search endpoints ===")

    # Try the main search endpoint with POST
    search_results = try_post_search_with_correct_format()

    if not search_results:
        print("\n=== Trying speciesSearch with POST ===")
        search_results = try_species_search_post()

    if not search_results:
        print("\n=== Examining working taxon for location structure ===")
        taxon_data = explore_working_taxon_for_clues()

        if taxon_data:
            print("\nNow we understand the data structure better!")

=== Trying POST requests to search endpoints ===

=== Trying POST payload 1 ===
Payload: {
  "criteriaType": "SpeciesSearchCriteria",
  "subnation": "US-SC",
  "globalRank": [
    "G1",
    "G2",
    "G3"
  ]
}
Status: 400
Error: {"timestamp":"2025-06-10T12:45:48.174+00:00","status":400,"error":"Bad Request","message":"JSON parse error: Could not resolve type id 'SpeciesSearchCriteria' as a subtype of `org.natureserve.nsx.search.criteria.CommonSearchCriteria`: known type ids = [combined, ecosystems, species]; nested exceptio

=== Trying POST payload 2 ===
Payload: {
  "criteriaType": "CommonSearchCriteria",
  "locationName": "South Carolina",
  "conservationConcern": true
}
Status: 400
Error: {"timestamp":"2025-06-10T12:45:48.274+00:00","status":400,"error":"Bad Request","message":"JSON parse error: Could not resolve type id 'CommonSearchCriteria' as a subtype of `org.natureserve.nsx.search.criteria.CommonSearchCriteria`: known type ids = [combined, ecosystems, species]; nested excepti

In [14]:
import requests
import json

def search_sc_endangered_species():
    """Search for endangered species in South Carolina using the correct API format"""

    search_url = "https://explorer.natureserve.org/api/data/search"

    headers = {
        'Content-Type': 'application/json',
        'Accept': 'application/json',
        'User-Agent': 'Mozilla/5.0 (compatible; research bot)'
    }

    # Based on the error message, valid criteriaType values are: combined, ecosystems, species
    # Based on the taxon data, South Carolina is referenced as:
    # - subnation.subnationCode: "SC"
    # - subnation.nameEn: "South Carolina"

    payloads_to_try = [
        {
            "criteriaType": "species",
            "subnationCodes": ["SC"],
            "globalRanks": ["G1", "G2", "G3"]
        },
        {
            "criteriaType": "species",
            "locationCriteria": {
                "subnationCodes": ["SC"]
            },
            "conservationCriteria": {
                "globalRanks": ["G1", "G2"]
            }
        },
        {
            "criteriaType": "species",
            "subnation": "SC",
            "globalRank": ["G1", "G2", "G3"]
        },
        {
            "criteriaType": "combined",
            "subnationCodes": ["SC"],
            "taxonCriteria": {
                "globalRanks": ["G1", "G2"]
            }
        }
    ]

    for i, payload in enumerate(payloads_to_try):
        try:
            print(f"\n=== Trying species search payload {i+1} ===")
            print(f"Payload: {json.dumps(payload, indent=2)}")

            response = requests.post(search_url, json=payload, headers=headers, timeout=20)
            print(f"Status: {response.status_code}")

            if response.status_code == 200:
                print("SUCCESS!")
                data = response.json()

                # Handle different response formats
                results = data.get('results', data.get('species', data if isinstance(data, list) else []))

                print(f"Found {len(results)} species")

                if results:
                    print("\nEndangered species in South Carolina:")
                    species_with_ids = []

                    for j, species in enumerate(results[:10]):  # Show first 10
                        sci_name = species.get('scientificName', 'Unknown')
                        common_name = species.get('commonName', species.get('primaryCommonName', 'No common name'))
                        global_rank = species.get('globalRank', species.get('roundedGlobalRank', 'Unknown'))
                        element_id = species.get('elementGlobalId', species.get('id'))

                        print(f"  {j+1}. {sci_name}")
                        print(f"     Common: {common_name}")
                        print(f"     Global Rank: {global_rank}")
                        print(f"     Element ID: {element_id}")
                        print()

                        if element_id:
                            species_with_ids.append(element_id)

                    return species_with_ids, results
                else:
                    print("No results in response")

            else:
                error_text = response.text[:300]
                print(f"Error: {error_text}")

        except Exception as e:
            print(f"Request error: {e}")

    return None, None

def get_species_geometries(element_ids):
    """Get detailed geometry information for specific species"""

    base_url = "https://explorer.natureserve.org/api/data/taxon/"

    headers = {'Accept': 'application/json'}

    species_with_geometry = []

    for element_id in element_ids[:5]:  # Limit to first 5 for now
        try:
            print(f"\n=== Getting geometry for element {element_id} ===")

            url = f"{base_url}{element_id}"
            response = requests.get(url, headers=headers, timeout=15)

            if response.status_code == 200:
                data = response.json()

                sci_name = data.get('scientificName', 'Unknown')
                common_name = data.get('primaryCommonName', 'Unknown')
                global_rank = data.get('roundedGlobalRank', 'Unknown')

                print(f"Species: {sci_name}")
                print(f"Common: {common_name}")
                print(f"Global Rank: {global_rank}")

                # Look for South Carolina specific information
                sc_info = []

                # Check elementNationals -> elementSubnationals for SC data
                element_nationals = data.get('elementNationals', [])
                for national in element_nationals:
                    element_subnationals = national.get('elementSubnationals', [])
                    for subnational in element_subnationals:
                        subnation = subnational.get('subnation', {})
                        if subnation.get('subnationCode') == 'SC':
                            sc_info.append({
                                'state_rank': subnational.get('subnationalRank'),
                                'rounded_state_rank': subnational.get('roundedSubnationalRank'),
                                'last_observed': subnational.get('lastObservedDate'),
                                'subnation_info': subnation
                            })

                if sc_info:
                    print(f"South Carolina Info:")
                    for info in sc_info:
                        print(f"  State Rank: {info['state_rank']}")
                        print(f"  Last Observed: {info['last_observed']}")

                # Look for geometry/mapping information
                geometry_sources = []

                # Check for map services (like the explorer-maps URLs we found earlier)
                if 'elementGlobalId' in data:
                    # Construct potential map service URLs based on the pattern we found
                    map_service_url = f"https://explorer.natureserve.org/explorer-maps/species_subnational_ranks/{data['elementGlobalId']}/FeatureServer/"
                    geometry_sources.append(('Map Service', map_service_url))

                # Look for other geometry fields
                geometry_fields = ['geometry', 'range', 'distribution', 'occurrences', 'rangeMap']
                for field in geometry_fields:
                    if field in data and data[field]:
                        geometry_sources.append((field, data[field]))

                if geometry_sources:
                    print(f"Geometry Sources Found:")
                    for source_type, source_data in geometry_sources:
                        print(f"  {source_type}: {str(source_data)[:100]}...")
                else:
                    print("No direct geometry found in taxon data")

                species_with_geometry.append({
                    'elementGlobalId': element_id,
                    'scientificName': sci_name,
                    'commonName': common_name,
                    'globalRank': global_rank,
                    'southCarolinaInfo': sc_info,
                    'geometrySources': geometry_sources,
                    'fullData': data
                })

                print("-" * 50)

        except Exception as e:
            print(f"Error getting geometry for {element_id}: {e}")

    return species_with_geometry

def try_map_service_geometry(element_id):
    """Try to get actual geometry from the map service"""

    # Based on the pattern we found: explorer-maps/species_subnational_ranks/{id}/FeatureServer/
    map_service_url = f"https://explorer.natureserve.org/explorer-maps/species_subnational_ranks/{element_id}/FeatureServer/0/query"

    headers = {'Accept': 'application/json'}

    # Common ArcGIS REST API parameters
    params = {
        'where': "1=1",  # Get all features
        'outFields': "*",  # Get all attributes
        'returnGeometry': "true",
        'f': "json"  # Return as JSON
    }

    try:
        print(f"\n=== Trying map service for element {element_id} ===")
        print(f"URL: {map_service_url}")

        response = requests.get(map_service_url, params=params, headers=headers, timeout=15)
        print(f"Status: {response.status_code}")

        if response.status_code == 200:
            data = response.json()

            if 'features' in data:
                features = data['features']
                print(f"Found {len(features)} geographic features")

                # Look for South Carolina features
                sc_features = []
                for feature in features:
                    attributes = feature.get('attributes', {})
                    geometry = feature.get('geometry', {})

                    # Look for SC in attributes
                    for key, value in attributes.items():
                        if isinstance(value, str) and ('sc' in value.lower() or 'south carolina' in value.lower()):
                            sc_features.append({
                                'attributes': attributes,
                                'geometry': geometry
                            })
                            break

                if sc_features:
                    print(f"Found {len(sc_features)} South Carolina features with geometry!")
                    for i, feature in enumerate(sc_features[:2]):
                        print(f"  Feature {i+1}:")
                        print(f"    Attributes: {feature['attributes']}")
                        print(f"    Geometry type: {feature['geometry'].get('type', 'Unknown')}")
                        if 'coordinates' in feature['geometry']:
                            print(f"    Coordinates preview: {str(feature['geometry']['coordinates'])[:100]}...")

                return sc_features
            else:
                print(f"No features found. Response keys: {list(data.keys())}")

        else:
            print(f"Error: {response.text[:200]}")

    except Exception as e:
        print(f"Map service error: {e}")

    return None

if __name__ == "__main__":
    print("=== Searching for endangered species in South Carolina ===")

    # Search for species
    element_ids, species_data = search_sc_endangered_species()

    if element_ids:
        print(f"\n=== Getting detailed geometry for {len(element_ids)} species ===")

        # Get detailed taxon information
        species_with_geometry = get_species_geometries(element_ids)

        # Try map service for first species
        if element_ids:
            map_features = try_map_service_geometry(element_ids[0])

        # Save results
        if species_with_geometry:
            with open('sc_endangered_species_with_geometry.json', 'w') as f:
                json.dump(species_with_geometry, f, indent=2, default=str)
            print(f"\nSaved {len(species_with_geometry)} species to sc_endangered_species_with_geometry.json")

    else:
        print("No endangered species found for South Carolina")

=== Searching for endangered species in South Carolina ===

=== Trying species search payload 1 ===
Payload: {
  "criteriaType": "species",
  "subnationCodes": [
    "SC"
  ],
  "globalRanks": [
    "G1",
    "G2",
    "G3"
  ]
}
Status: 400
Error: {"timestamp":"2025-06-10T12:46:52.121+00:00","status":400,"error":"Bad Request","message":"JSON parse error: Could not resolve type id 'species' as a subtype of `org.natureserve.nsx.search.criteria.CommonSearchCriteria`: Class `org.natureserve.nsx.search.criteria.SpeciesSearchCriteria` not subtype o

=== Trying species search payload 2 ===
Payload: {
  "criteriaType": "species",
  "locationCriteria": {
    "subnationCodes": [
      "SC"
    ]
  },
  "conservationCriteria": {
    "globalRanks": [
      "G1",
      "G2"
    ]
  }
}
Status: 400
Error: {"timestamp":"2025-06-10T12:46:52.291+00:00","status":400,"error":"Bad Request","message":"JSON parse error: Could not resolve type id 'species' as a subtype of `org.natureserve.nsx.search.criteri

In [15]:
import requests
import json
import time

def try_minimal_search_formats():
    """Try the most minimal possible search formats"""

    search_url = "https://explorer.natureserve.org/api/data/search"

    headers = {
        'Content-Type': 'application/json',
        'Accept': 'application/json'
    }

    # Try extremely minimal payloads
    minimal_payloads = [
        # Just the criteriaType
        {"criteriaType": "combined"},
        {"criteriaType": "ecosystems"},
        {"criteriaType": "species"},

        # Try with just basic fields
        {
            "criteriaType": "combined",
            "text": "South Carolina"
        },
        {
            "criteriaType": "combined",
            "searchText": "South Carolina"
        },

        # Try empty criteria to see what fields are expected
        {
            "criteriaType": "combined",
            "criteria": {}
        }
    ]

    for i, payload in enumerate(minimal_payloads):
        try:
            print(f"\n=== Minimal payload {i+1}: {json.dumps(payload)} ===")

            response = requests.post(search_url, json=payload, headers=headers, timeout=15)
            print(f"Status: {response.status_code}")

            if response.status_code == 200:
                print("SUCCESS! This format works!")
                data = response.json()
                print(f"Response preview: {json.dumps(data, indent=2)[:300]}...")
                return payload, data
            else:
                error = response.text[:200]
                print(f"Error: {error}")

                # Look for clues about expected fields in error messages
                if "Unrecognized field" in error:
                    print("  ^ This tells us about invalid field names")
                elif "missing" in error.lower():
                    print("  ^ This tells us about required fields")

        except Exception as e:
            print(f"Request error: {e}")

    return None, None

def try_speciesSearch_endpoint():
    """Try the speciesSearch endpoint directly with minimal payload"""

    species_search_url = "https://explorer.natureserve.org/api/data/speciesSearch"

    headers = {
        'Content-Type': 'application/json',
        'Accept': 'application/json'
    }

    # Try minimal species search payloads
    species_payloads = [
        {},  # Empty to see what's required
        {"searchText": "endangered"},
        {"q": "South Carolina"},
        {"location": "South Carolina"},
        {"criteriaType": "species"}
    ]

    for i, payload in enumerate(species_payloads):
        try:
            print(f"\n=== SpeciesSearch payload {i+1}: {json.dumps(payload)} ===")

            response = requests.post(species_search_url, json=payload, headers=headers, timeout=15)
            print(f"Status: {response.status_code}")

            if response.status_code == 200:
                print("SUCCESS!")
                data = response.json()
                results = data.get('results', data.get('species', []))
                print(f"Found {len(results)} results")
                return payload, data
            else:
                print(f"Error: {response.text[:200]}")

        except Exception as e:
            print(f"Error: {e}")

    return None, None

def try_direct_taxon_search():
    """Since individual taxon lookups work, try to find SC species by searching known IDs"""

    # We know this ID works: ELEMENT_GLOBAL.2.154701
    # Let's try some other IDs that might be in South Carolina

    base_url = "https://explorer.natureserve.org/api/data/taxon/"
    headers = {'Accept': 'application/json'}

    # Try a range of element IDs to find more SC species
    sc_species = []

    # Start with the known working ID and try nearby ones
    test_ids = [154701, 154702, 154703, 154700, 154699, 102187]  # 102187 was in the explorer-maps URL

    for element_id in test_ids:
        try:
            full_id = f"ELEMENT_GLOBAL.2.{element_id}"
            url = f"{base_url}{full_id}"

            print(f"Checking {full_id}...")
            response = requests.get(url, headers=headers, timeout=10)

            if response.status_code == 200:
                data = response.json()

                # Check if this species occurs in South Carolina
                has_sc = False
                sci_name = data.get('scientificName', 'Unknown')
                global_rank = data.get('roundedGlobalRank', 'Unknown')

                # Check elementNationals for SC
                element_nationals = data.get('elementNationals', [])
                sc_ranks = []

                for national in element_nationals:
                    element_subnationals = national.get('elementSubnationals', [])
                    for subnational in element_subnationals:
                        subnation = subnational.get('subnation', {})
                        if subnation.get('subnationCode') == 'SC':
                            has_sc = True
                            sc_ranks.append(subnational.get('roundedSubnationalRank', 'Unknown'))

                if has_sc:
                    print(f"  ✓ Found SC species: {sci_name}")
                    print(f"    Global Rank: {global_rank}")
                    print(f"    SC Ranks: {sc_ranks}")

                    # Check if it's endangered (G1, G2, G3 or S1, S2, S3)
                    is_endangered = (
                        global_rank in ['G1', 'G2', 'G3'] or
                        any(rank in ['S1', 'S2', 'S3'] for rank in sc_ranks)
                    )

                    if is_endangered:
                        print(f"    >>> ENDANGERED SPECIES FOUND! <<<")
                        sc_species.append({
                            'elementId': full_id,
                            'scientificName': sci_name,
                            'globalRank': global_rank,
                            'scRanks': sc_ranks,
                            'data': data
                        })

                    print()
                else:
                    print(f"  - {sci_name} (not in SC)")

            elif response.status_code == 404:
                print(f"  - {full_id} not found")
            else:
                print(f"  - {full_id} error: {response.status_code}")

            # Small delay to be nice to their server
            time.sleep(0.5)

        except Exception as e:
            print(f"  Error with {element_id}: {e}")

    return sc_species

def try_map_service_for_known_species():
    """Try the map service we found for the known species"""

    # We saw this in the HTML: explorer-maps/species_subnational_ranks/ELEMENT_GLOBAL.2.102187
    element_id = "ELEMENT_GLOBAL.2.102187"

    map_url = f"https://explorer.natureserve.org/explorer-maps/species_subnational_ranks/{element_id}/FeatureServer/0/query"

    params = {
        'where': "1=1",
        'outFields': "*",
        'returnGeometry': "true",
        'f': "json"
    }

    headers = {'Accept': 'application/json'}

    try:
        print(f"\n=== Trying map service for {element_id} ===")

        response = requests.get(map_url, params=params, headers=headers, timeout=15)
        print(f"Status: {response.status_code}")

        if response.status_code == 200:
            data = response.json()
            print(f"Response keys: {list(data.keys())}")

            if 'features' in data:
                features = data['features']
                print(f"Found {len(features)} features with geometry")

                # Look for South Carolina
                for i, feature in enumerate(features[:5]):
                    attributes = feature.get('attributes', {})
                    geometry = feature.get('geometry', {})

                    print(f"Feature {i+1}:")
                    print(f"  Attributes: {list(attributes.keys())}")
                    print(f"  Geometry type: {geometry.get('type')}")

                    # Check if this is South Carolina
                    for key, value in attributes.items():
                        if isinstance(value, str) and ('sc' in value.lower() or 'south carolina' in value.lower()):
                            print(f"  >>> SOUTH CAROLINA FEATURE FOUND! <<<")
                            print(f"  Attributes: {attributes}")
                            print(f"  Geometry: {json.dumps(geometry, indent=2)[:300]}...")
                            return feature

            return data
        else:
            print(f"Error: {response.text[:200]}")

    except Exception as e:
        print(f"Map service error: {e}")

    return None

if __name__ == "__main__":
    print("=== Trying minimal search formats ===")
    working_format, data = try_minimal_search_formats()

    if not working_format:
        print("\n=== Trying speciesSearch endpoint ===")
        working_format, data = try_speciesSearch_endpoint()

    if not working_format:
        print("\n=== Trying direct taxon ID search ===")
        sc_species = try_direct_taxon_search()

        if sc_species:
            print(f"\nFound {len(sc_species)} endangered species in South Carolina!")
            for species in sc_species:
                print(f"- {species['scientificName']} ({species['globalRank']})")

    print("\n=== Trying map service for known species ===")
    map_data = try_map_service_for_known_species()

=== Trying minimal search formats ===

=== Minimal payload 1: {"criteriaType": "combined"} ===
Status: 200
SUCCESS! This format works!
Response preview: {
  "results": [
    {
      "recordType": "SPECIES",
      "elementGlobalId": 828458,
      "uniqueId": "ELEMENT_GLOBAL.2.828458",
      "nsxUrl": "/Taxon/ELEMENT_GLOBAL.2.828458/Acris_blanchardi",
      "elcode": "AAABC01040",
      "scientificName": "Acris blanchardi",
      "formattedScientificN...

=== Trying map service for known species ===

=== Trying map service for ELEMENT_GLOBAL.2.102187 ===
Status: 200
Response keys: ['objectIdFieldName', 'uniqueIdField', 'globalIdFieldName', 'hasZ', 'hasM', 'spatialReference', 'fields', 'features', 'exceededTransferLimit', 'geometryType']
Found 28 features with geometry
Feature 1:
  Attributes: ['objid', 'name', 'country', 'admintype', 'taxon_unique_id', 'element_subnational_id', 'rounded_s_rank', 's_rank', 'exotic_ind', 'hybrid_ind', 'subnation_code', 'nation_code', 's_rank_viz', 'data_re

In [16]:
import requests
import json

def search_all_species_and_filter_sc():
    """Use the working search format and filter for SC endangered species"""

    search_url = "https://explorer.natureserve.org/api/data/search"

    headers = {
        'Content-Type': 'application/json',
        'Accept': 'application/json'
    }

    # Use the working format
    payload = {"criteriaType": "combined"}

    try:
        print("=== Getting all species from NatureServe ===")
        response = requests.post(search_url, json=payload, headers=headers, timeout=30)

        if response.status_code == 200:
            data = response.json()
            all_results = data.get('results', [])

            print(f"Total results: {len(all_results)}")

            # Filter for species only (not ecosystems)
            species_results = [r for r in all_results if r.get('recordType') == 'SPECIES']
            print(f"Species results: {len(species_results)}")

            # Now check each species to see if it occurs in SC and is endangered
            sc_endangered_species = []

            print("Checking species for South Carolina occurrence and conservation status...")

            for i, species in enumerate(species_results[:50]):  # Limit to first 50 for testing
                if i % 10 == 0:
                    print(f"  Checked {i} species...")

                element_id = species.get('uniqueId', species.get('elementGlobalId'))
                if element_id:
                    sc_species_data = check_species_for_sc_and_conservation(element_id)
                    if sc_species_data:
                        sc_endangered_species.append(sc_species_data)
                        print(f"  Found: {sc_species_data['scientificName']}")

            return sc_endangered_species

    except Exception as e:
        print(f"Search error: {e}")
        return None

def check_species_for_sc_and_conservation(element_id):
    """Check if a species occurs in SC and has conservation concern"""

    # Handle different ID formats
    if not element_id.startswith('ELEMENT_GLOBAL'):
        element_id = f"ELEMENT_GLOBAL.2.{element_id}"

    taxon_url = f"https://explorer.natureserve.org/api/data/taxon/{element_id}"
    headers = {'Accept': 'application/json'}

    try:
        response = requests.get(taxon_url, headers=headers, timeout=10)

        if response.status_code == 200:
            data = response.json()

            # Check for South Carolina occurrence
            sc_info = []
            element_nationals = data.get('elementNationals', [])

            for national in element_nationals:
                element_subnationals = national.get('elementSubnationals', [])
                for subnational in element_subnationals:
                    subnation = subnational.get('subnation', {})
                    if subnation.get('subnationCode') == 'SC':
                        sc_info.append({
                            'state_rank': subnational.get('subnationalRank'),
                            'rounded_state_rank': subnational.get('roundedSubnationalRank'),
                            'last_observed': subnational.get('lastObservedDate')
                        })

            if sc_info:
                # Check conservation status
                global_rank = data.get('roundedGlobalRank', '')

                # Check if endangered (G1, G2, G3 or S1, S2, S3)
                is_endangered = global_rank in ['G1', 'G2', 'G3']

                for info in sc_info:
                    state_rank = info.get('rounded_state_rank', '')
                    if state_rank in ['S1', 'S2', 'S3']:
                        is_endangered = True
                        break

                if is_endangered:
                    return {
                        'elementId': element_id,
                        'scientificName': data.get('scientificName', 'Unknown'),
                        'commonName': data.get('primaryCommonName', 'Unknown'),
                        'globalRank': global_rank,
                        'scInfo': sc_info,
                        'fullData': data
                    }

    except Exception as e:
        # Silently continue - many IDs won't exist
        pass

    return None

def get_geometry_from_map_service(element_id):
    """Get geometry for a specific species from the map service"""

    # Clean element ID for URL
    if element_id.startswith('ELEMENT_GLOBAL.2.'):
        clean_id = element_id
    else:
        clean_id = f"ELEMENT_GLOBAL.2.{element_id}"

    map_url = f"https://explorer.natureserve.org/explorer-maps/species_subnational_ranks/{clean_id}/FeatureServer/0/query"

    params = {
        'where': "subnation_code='SC'",  # Filter for South Carolina only
        'outFields': "*",
        'returnGeometry': "true",
        'f': "json"
    }

    headers = {'Accept': 'application/json'}

    try:
        response = requests.get(map_url, params=params, headers=headers, timeout=15)

        if response.status_code == 200:
            data = response.json()
            features = data.get('features', [])

            if features:
                print(f"  Found {len(features)} SC geometric features for {clean_id}")
                return features
            else:
                # Try without the SC filter to see if any geometry exists
                params['where'] = "1=1"
                response = requests.get(map_url, params=params, headers=headers, timeout=15)

                if response.status_code == 200:
                    data = response.json()
                    all_features = data.get('features', [])

                    # Filter for SC manually
                    sc_features = []
                    for feature in all_features:
                        attributes = feature.get('attributes', {})
                        if attributes.get('subnation_code') == 'SC':
                            sc_features.append(feature)

                    if sc_features:
                        print(f"  Found {len(sc_features)} SC features (manual filter) for {clean_id}")
                        return sc_features

    except Exception as e:
        print(f"  Map service error for {element_id}: {e}")

    return None

def build_complete_sc_endangered_dataset():
    """Build complete dataset of SC endangered species with geometries"""

    print("=== Building Complete South Carolina Endangered Species Dataset ===")

    # First, get endangered species in SC
    sc_species = search_all_species_and_filter_sc()

    if not sc_species:
        print("No endangered species found")
        return None

    print(f"\nFound {len(sc_species)} endangered species in South Carolina")
    print("Now getting geometries...")

    # Add geometries to each species
    complete_dataset = []

    for species in sc_species:
        print(f"\nProcessing: {species['scientificName']}")

        # Get geometry from map service
        geometry_features = get_geometry_from_map_service(species['elementId'])

        species_with_geometry = {
            'scientificName': species['scientificName'],
            'commonName': species['commonName'],
            'elementId': species['elementId'],
            'globalRank': species['globalRank'],
            'southCarolinaInfo': species['scInfo'],
            'geometryFeatures': geometry_features,
            'hasGeometry': bool(geometry_features)
        }

        complete_dataset.append(species_with_geometry)

        if geometry_features:
            print(f"  ✓ Has geometry ({len(geometry_features)} features)")
        else:
            print(f"  ✗ No geometry found")

    # Save complete dataset
    with open('sc_endangered_species_complete.json', 'w') as f:
        json.dump(complete_dataset, f, indent=2, default=str)

    print(f"\n=== SUMMARY ===")
    print(f"Total endangered species in SC: {len(complete_dataset)}")
    species_with_geometry = [s for s in complete_dataset if s['hasGeometry']]
    print(f"Species with geometry: {len(species_with_geometry)}")

    print(f"\nSpecies with geometry:")
    for species in species_with_geometry:
        print(f"- {species['scientificName']} ({species['globalRank']})")

    print(f"\nDataset saved to: sc_endangered_species_complete.json")

    return complete_dataset

if __name__ == "__main__":
    # Build the complete dataset
    dataset = build_complete_sc_endangered_dataset()

    if dataset:
        print("\n" + "="*60)
        print("SUCCESS! You now have endangered species geometries for South Carolina!")
        print("="*60)

=== Building Complete South Carolina Endangered Species Dataset ===
=== Getting all species from NatureServe ===
Total results: 20
Species results: 20
Checking species for South Carolina occurrence and conservation status...
  Checked 0 species...
  Checked 10 species...
No endangered species found


In [17]:
import requests
import json
import time

def try_pagination_and_filters():
    """Try to get more results with pagination and different filters"""

    search_url = "https://explorer.natureserve.org/api/data/search"

    headers = {
        'Content-Type': 'application/json',
        'Accept': 'application/json'
    }

    # Try different payload variations to get more results
    payloads_to_try = [
        # Basic with pagination
        {
            "criteriaType": "combined",
            "pageSize": 100,
            "pageNumber": 0
        },
        {
            "criteriaType": "combined",
            "limit": 1000
        },
        {
            "criteriaType": "combined",
            "maxResults": 1000
        },
        # Try text search for South Carolina
        {
            "criteriaType": "combined",
            "searchText": "South Carolina"
        },
        {
            "criteriaType": "combined",
            "locationText": "South Carolina"
        },
        {
            "criteriaType": "combined",
            "textCriteria": "South Carolina"
        }
    ]

    for i, payload in enumerate(payloads_to_try):
        try:
            print(f"\n=== Trying payload {i+1}: {json.dumps(payload)} ===")

            response = requests.post(search_url, json=payload, headers=headers, timeout=30)
            print(f"Status: {response.status_code}")

            if response.status_code == 200:
                data = response.json()
                results = data.get('results', [])

                print(f"Got {len(results)} results")

                if len(results) > 20:
                    print("SUCCESS! Found more results")
                    return results
                elif len(results) > 0:
                    print("Sample results:")
                    for j, result in enumerate(results[:3]):
                        print(f"  {j+1}. {result.get('scientificName', 'Unknown')} ({result.get('recordType', 'Unknown')})")

            else:
                print(f"Error: {response.text[:200]}")

        except Exception as e:
            print(f"Error: {e}")

    return None

def try_known_endangered_species_ids():
    """Try some known endangered species element IDs"""

    # Based on common endangered species, let's try some likely IDs
    # These are educated guesses based on typical NatureServe patterns

    base_url = "https://explorer.natureserve.org/api/data/taxon/"
    headers = {'Accept': 'application/json'}

    # Try a wider range of IDs - endangered species often have lower numbers
    id_ranges = [
        range(100000, 100100),  # Early range
        range(150000, 150100),  # Around the working ID we found
        range(200000, 200100),  # Higher range
        range(102180, 102200),  # Around the map service ID we found
    ]

    sc_endangered_species = []

    for id_range in id_ranges:
        print(f"\nChecking ID range {id_range.start} to {id_range.stop-1}...")

        for element_num in id_range:
            element_id = f"ELEMENT_GLOBAL.2.{element_num}"

            try:
                response = requests.get(f"{base_url}{element_id}", headers=headers, timeout=5)

                if response.status_code == 200:
                    data = response.json()

                    # Quick check for SC and endangered status
                    sci_name = data.get('scientificName', '')
                    global_rank = data.get('roundedGlobalRank', '')

                    # Check for SC occurrence
                    has_sc = False
                    sc_ranks = []

                    element_nationals = data.get('elementNationals', [])
                    for national in element_nationals:
                        element_subnationals = national.get('elementSubnationals', [])
                        for subnational in element_subnationals:
                            subnation = subnational.get('subnation', {})
                            if subnation.get('subnationCode') == 'SC':
                                has_sc = True
                                sc_rank = subnational.get('roundedSubnationalRank', '')
                                if sc_rank:
                                    sc_ranks.append(sc_rank)

                    if has_sc:
                        # Check if endangered
                        is_endangered = (
                            global_rank in ['G1', 'G2', 'G3'] or
                            any(rank in ['S1', 'S2', 'S3'] for rank in sc_ranks)
                        )

                        if is_endangered:
                            print(f"  ✓ FOUND: {sci_name} ({global_rank}, SC: {sc_ranks})")
                            sc_endangered_species.append({
                                'elementId': element_id,
                                'scientificName': sci_name,
                                'globalRank': global_rank,
                                'scRanks': sc_ranks,
                                'data': data
                            })
                        else:
                            print(f"  - {sci_name} (in SC but not endangered)")

                # Small delay to be respectful
                time.sleep(0.1)

            except:
                pass  # Silently continue for 404s and other errors

        if len(sc_endangered_species) >= 10:
            print(f"Found {len(sc_endangered_species)} species, stopping search...")
            break

    return sc_endangered_species

def get_geometries_for_species_list(species_list):
    """Get geometries for a list of species"""

    species_with_geometries = []

    for species in species_list:
        print(f"\nGetting geometry for: {species['scientificName']}")

        # Try the map service
        geometry_features = get_geometry_from_map_service(species['elementId'])

        species_record = {
            'scientificName': species['scientificName'],
            'elementId': species['elementId'],
            'globalRank': species['globalRank'],
            'scRanks': species.get('scRanks', []),
            'geometryFeatures': geometry_features,
            'hasGeometry': bool(geometry_features)
        }

        species_with_geometries.append(species_record)

        if geometry_features:
            print(f"  ✓ Found {len(geometry_features)} geometric features")

            # Show sample geometry
            for i, feature in enumerate(geometry_features[:1]):
                attrs = feature.get('attributes', {})
                geom = feature.get('geometry', {})
                print(f"    Feature {i+1}: {attrs.get('name', 'Unknown')} - {geom.get('type', 'No geometry type')}")
        else:
            print(f"  ✗ No geometry found")

    return species_with_geometries

def get_geometry_from_map_service(element_id):
    """Get geometry for a specific species from the map service"""

    clean_id = element_id
    map_url = f"https://explorer.natureserve.org/explorer-maps/species_subnational_ranks/{clean_id}/FeatureServer/0/query"

    params = {
        'where': "subnation_code='SC'",
        'outFields': "*",
        'returnGeometry': "true",
        'f': "json"
    }

    headers = {'Accept': 'application/json'}

    try:
        response = requests.get(map_url, params=params, headers=headers, timeout=10)

        if response.status_code == 200:
            data = response.json()
            features = data.get('features', [])

            if features:
                return features

            # Try without SC filter
            params['where'] = "1=1"
            response = requests.get(map_url, params=params, headers=headers, timeout=10)

            if response.status_code == 200:
                data = response.json()
                all_features = data.get('features', [])

                # Filter for SC
                sc_features = []
                for feature in all_features:
                    attrs = feature.get('attributes', {})
                    if attrs.get('subnation_code') == 'SC':
                        sc_features.append(feature)

                return sc_features

    except Exception as e:
        pass

    return None

if __name__ == "__main__":
    print("=== Trying to get more comprehensive results ===")

    # First try different search approaches
    more_results = try_pagination_and_filters()

    if not more_results or len(more_results) <= 20:
        print("\n=== Trying direct ID search for endangered species ===")
        sc_species = try_known_endangered_species_ids()

        if sc_species:
            print(f"\nFound {len(sc_species)} endangered species in South Carolina:")
            for species in sc_species:
                print(f"- {species['scientificName']} ({species['globalRank']})")

            print(f"\n=== Getting geometries ===")
            complete_dataset = get_geometries_for_species_list(sc_species)

            # Save results
            with open('sc_endangered_species_with_geometries.json', 'w') as f:
                json.dump(complete_dataset, f, indent=2, default=str)

            # Summary
            with_geometry = [s for s in complete_dataset if s['hasGeometry']]

            print(f"\n=== FINAL RESULTS ===")
            print(f"Total endangered species found: {len(complete_dataset)}")
            print(f"Species with geometry data: {len(with_geometry)}")

            if with_geometry:
                print(f"\nSpecies with geometries:")
                for species in with_geometry:
                    print(f"- {species['scientificName']} ({species['globalRank']})")

                print(f"\nData saved to: sc_endangered_species_with_geometries.json")
                print("SUCCESS!")
            else:
                print("No species found with geometry data")
        else:
            print("No endangered species found in ID search")
    else:
        print(f"Found {len(more_results)} results to process...")

=== Trying to get more comprehensive results ===

=== Trying payload 1: {"criteriaType": "combined", "pageSize": 100, "pageNumber": 0} ===
Status: 400
Error: {"timestamp":"2025-06-10T12:49:48.061+00:00","status":400,"error":"Bad Request","message":"JSON parse error: Unrecognized field \"pageSize\" (class org.natureserve.nsx.search.criteria.CommonSearchCrit

=== Trying payload 2: {"criteriaType": "combined", "limit": 1000} ===
Status: 400
Error: {"timestamp":"2025-06-10T12:49:48.165+00:00","status":400,"error":"Bad Request","message":"JSON parse error: Unrecognized field \"limit\" (class org.natureserve.nsx.search.criteria.CommonSearchCriteri

=== Trying payload 3: {"criteriaType": "combined", "maxResults": 1000} ===
Status: 400
Error: {"timestamp":"2025-06-10T12:49:48.275+00:00","status":400,"error":"Bad Request","message":"JSON parse error: Unrecognized field \"maxResults\" (class org.natureserve.nsx.search.criteria.CommonSearchCr

=== Trying payload 4: {"criteriaType": "combined", "s

In [19]:
import requests
import json
import time

def search_for_specific_endangered_species():
    """Search for specific endangered species known to occur in South Carolina"""

    # List of known endangered species in South Carolina with their likely element IDs
    # Based on research of federal and state endangered species lists
    endangered_species_info = [
        # Federally Endangered Birds
        {"name": "Picoides borealis", "common": "Red-cockaded woodpecker", "status": "Endangered"},

        # Marine Species
        {"name": "Dermochelys coriacea", "common": "Leatherback sea turtle", "status": "Endangered"},
        {"name": "Lepidochelys kempii", "common": "Kemp's Ridley sea turtle", "status": "Endangered"},
        {"name": "Caretta caretta", "common": "Loggerhead sea turtle", "status": "Threatened"},
        {"name": "Eubalaena glacialis", "common": "North Atlantic right whale", "status": "Endangered"},

        # Mammals
        {"name": "Myotis sodalis", "common": "Indiana bat", "status": "Endangered"},
        {"name": "Myotis septentrionalis", "common": "Northern long-eared bat", "status": "Endangered"},
        {"name": "Corynorhinus rafinesquii", "common": "Rafinesque's big-eared bat", "status": "Endangered"},
        {"name": "Trichechus manatus", "common": "West-Indian manatee", "status": "Threatened"},

        # Fish
        {"name": "Acipenser oxyrinchus", "common": "Atlantic sturgeon", "status": "Endangered"},
        {"name": "Acipenser brevirostrum", "common": "Shortnose sturgeon", "status": "Endangered"},

        # Invertebrates
        {"name": "Lasmigona decorata", "common": "Carolina heelsplitter", "status": "Endangered"},
        {"name": "Bombus affinis", "common": "Rusty-patched bumble bee", "status": "Endangered"},

        # Reptiles and Amphibians
        {"name": "Glyptemys muhlenbergii", "common": "Bog turtle", "status": "Threatened"},
        {"name": "Drymarchon couperi", "common": "Eastern indigo snake", "status": "Threatened"},
        {"name": "Ambystoma cingulatum", "common": "Flatwoods salamander", "status": "Threatened"},
        {"name": "Gopherus polyphemus", "common": "Gopher tortoise", "status": "Threatened"},

        {"name": "Charadrius melodus", "common": "Piping plover", "status": "Threatened"},
        {"name": "Calidris canutus", "common": "Red knot", "status": "Threatened"},
        {"name": "Mycteria americana", "common": "Wood stork", "status": "Threatened"},

        # Plants
        {"name": "Helianthus schweinitzii", "common": "Schweinitz sunflower", "status": "Endangered"},
        {"name": "Amaranthus pumilus", "common": "Seabeach amaranth", "status": "Threatened"},
        {"name": "Isotria medeoloides", "common": "Small whorled pogonia", "status": "Threatened"}
    ]

    # Since the NatureServe search only returns 20 random species, we'll try a direct approach
    # by searching for these species by trying common element ID patterns

    found_species = []

    print("Searching for known endangered species in South Carolina...")
    print("Using direct species lookup approach...")
    print("=" * 60)

    # Try to find these species by searching element ID ranges and scientific names
    for species_info in endangered_species_info:
        sci_name = species_info["name"]
        common_name = species_info["common"]
        status = species_info["status"]

        print(f"Searching for: {sci_name} ({common_name})")

        # Try multiple approaches to find this species
        species_data = find_species_multiple_methods(sci_name, common_name, status)

        if species_data:
            found_species.append(species_data)
            print(f"  ✓ Found and confirmed in South Carolina")
        else:
            print(f"  - Not found in current search scope")
        print()

    return found_species

def find_species_multiple_methods(sci_name, common_name, status):
    """Try multiple methods to find a species in NatureServe"""

    # Method 1: Try the general search (though it only returns 20 species)
    species_data = search_in_general_results(sci_name)
    if species_data:
        return species_data

    # Method 2: Try common element ID patterns for this type of species
    species_data = try_element_id_patterns(sci_name, common_name)
    if species_data:
        return species_data

    # Method 3: Could add more methods here (web scraping the website, etc.)

    return None

def search_in_general_results(sci_name):
    """Search for species in the general 20-species result set"""

    search_url = "https://explorer.natureserve.org/api/data/search"
    search_headers = {
        'Content-Type': 'application/json',
        'Accept': 'application/json'
    }

    try:
        response = requests.post(search_url, json={"criteriaType": "combined"},
                               headers=search_headers, timeout=15)

        if response.status_code == 200:
            data = response.json()
            all_species = data.get('results', [])

            # Look for our target species
            for species in all_species:
                if species.get('scientificName') == sci_name:
                    element_id = species.get('uniqueId', species.get('elementGlobalId'))
                    if element_id:
                        return check_species_for_sc_occurrence(element_id, sci_name)
    except:
        pass

    return None

def try_element_id_patterns(sci_name, common_name):
    """Try common element ID patterns to find species directly"""

    # This is a more systematic approach since the search API is limited
    # We'll try ranges of IDs that are likely to contain endangered species

    id_ranges_to_try = [
        range(100000, 100200),  # Early IDs often contain well-known species
        range(102000, 102300),  # Around the working map service ID we found
        range(150000, 150200),  # Around the goldenseal ID that worked
        range(200000, 200100),  # Higher range
        range(828000, 829000),  # Around the first ID from our successful search
    ]

    for id_range in id_ranges_to_try:
        for element_num in id_range:
            element_id = f"ELEMENT_GLOBAL.2.{element_num}"

            species_data = check_if_target_species(element_id, sci_name)
            if species_data:
                return species_data

            # Small delay to be respectful of their servers
            time.sleep(0.05)

    return None

def check_if_target_species(element_id, target_sci_name):
    """Check if a given element ID matches our target species and occurs in SC"""

    taxon_url = f"https://explorer.natureserve.org/api/data/taxon/{element_id}"
    headers = {'Accept': 'application/json'}

    try:
        response = requests.get(taxon_url, headers=headers, timeout=5)

        if response.status_code == 200:
            data = response.json()
            sci_name = data.get('scientificName', '')

            # Check if this is our target species
            if sci_name == target_sci_name:
                # Check if it occurs in South Carolina
                return check_species_for_sc_occurrence_from_data(data, element_id)
    except:
        pass

    return None

def check_species_for_sc_occurrence_from_data(data, element_id):
    """Check SC occurrence from already-loaded species data"""

    # Check for South Carolina occurrence
    sc_info = []
    element_nationals = data.get('elementNationals', [])

    for national in element_nationals:
        element_subnationals = national.get('elementSubnationals', [])
        for subnational in element_subnationals:
            subnation = subnational.get('subnation', {})
            if subnation.get('subnationCode') == 'SC':
                sc_info.append({
                    'state_rank': subnational.get('subnationalRank'),
                    'rounded_state_rank': subnational.get('roundedSubnationalRank'),
                    'last_observed': subnational.get('lastObservedDate'),
                    'subnation_name': subnation.get('nameEn', 'South Carolina')
                })

    if sc_info:
        return {
            'elementId': element_id,
            'scientificName': data.get('scientificName'),
            'commonName': data.get('primaryCommonName', 'Unknown'),
            'globalRank': data.get('roundedGlobalRank', 'Unknown'),
            'scInfo': sc_info,
            'fullData': data
        }

    return None

    base_url = "https://explorer.natureserve.org/api/data/taxon/"
    headers = {'Accept': 'application/json'}

    found_species = []

    print("Searching for known endangered species in South Carolina...")
    print("=" * 60)

    # We'll use the working search to get all species, then filter
    search_url = "https://explorer.natureserve.org/api/data/search"
    search_headers = {
        'Content-Type': 'application/json',
        'Accept': 'application/json'
    }

    try:
        # Get all species from the working search
        response = requests.post(search_url, json={"criteriaType": "combined"}, headers=search_headers, timeout=30)

        if response.status_code == 200:
            data = response.json()
            all_species = data.get('results', [])

            print(f"Got {len(all_species)} total species from NatureServe")
            print("Filtering for endangered species in South Carolina...\n")

            # Check each species to see if it's on our endangered list and occurs in SC
            for species in all_species:
                sci_name = species.get('scientificName', '')
                element_id = species.get('uniqueId', species.get('elementGlobalId'))

                # Check if this is one of our target endangered species
                if sci_name in endangered_species_names:
                    print(f"Found target species: {sci_name}")

                    # Get detailed information
                    if element_id:
                        sc_species_data = check_species_for_sc_occurrence(element_id, sci_name)
                        if sc_species_data:
                            found_species.append(sc_species_data)
                            print(f"  ✓ Confirmed in South Carolina")
                        else:
                            print(f"  - Not found in South Carolina")
                    print()

            return found_species

    except Exception as e:
        print(f"Search error: {e}")
        return []

def check_species_for_sc_occurrence(element_id, sci_name):
    """Check if a species occurs in South Carolina and get its conservation status"""

    # Handle different ID formats
    if not str(element_id).startswith('ELEMENT_GLOBAL'):
        element_id = f"ELEMENT_GLOBAL.2.{element_id}"

    taxon_url = f"https://explorer.natureserve.org/api/data/taxon/{element_id}"
    headers = {'Accept': 'application/json'}

    try:
        response = requests.get(taxon_url, headers=headers, timeout=10)

        if response.status_code == 200:
            data = response.json()

            # Check for South Carolina occurrence
            sc_info = []
            element_nationals = data.get('elementNationals', [])

            for national in element_nationals:
                element_subnationals = national.get('elementSubnationals', [])
                for subnational in element_subnationals:
                    subnation = subnational.get('subnation', {})
                    if subnation.get('subnationCode') == 'SC':
                        sc_info.append({
                            'state_rank': subnational.get('subnationalRank'),
                            'rounded_state_rank': subnational.get('roundedSubnationalRank'),
                            'last_observed': subnational.get('lastObservedDate'),
                            'subnation_name': subnation.get('nameEn', 'South Carolina')
                        })

            if sc_info:
                return {
                    'elementId': element_id,
                    'scientificName': sci_name,
                    'commonName': data.get('primaryCommonName', 'Unknown'),
                    'globalRank': data.get('roundedGlobalRank', 'Unknown'),
                    'scInfo': sc_info,
                    'fullData': data
                }

    except Exception as e:
        print(f"    Error checking {sci_name}: {e}")

    return None

def get_geometry_from_map_service(element_id, species_name):
    """Get geometry for a specific species from the map service"""

    clean_id = element_id
    map_url = f"https://explorer.natureserve.org/explorer-maps/species_subnational_ranks/{clean_id}/FeatureServer/0/query"

    params = {
        'where': "subnation_code='SC'",
        'outFields': "*",
        'returnGeometry': "true",
        'f': "json"
    }

    headers = {'Accept': 'application/json'}

    try:
        print(f"  Trying map service for {species_name}...")
        response = requests.get(map_url, params=params, headers=headers, timeout=15)

        if response.status_code == 200:
            data = response.json()
            features = data.get('features', [])

            if features:
                print(f"    ✓ Found {len(features)} SC geometric features")
                return features
            else:
                # Try without the SC filter to see if any geometry exists
                params['where'] = "1=1"
                response = requests.get(map_url, params=params, headers=headers, timeout=15)

                if response.status_code == 200:
                    data = response.json()
                    all_features = data.get('features', [])

                    # Filter for SC manually
                    sc_features = []
                    for feature in all_features:
                        attributes = feature.get('attributes', {})
                        if attributes.get('subnation_code') == 'SC':
                            sc_features.append(feature)

                    if sc_features:
                        print(f"    ✓ Found {len(sc_features)} SC features (manual filter)")
                        return sc_features
                    else:
                        print(f"    - No SC-specific geometry found")
                else:
                    print(f"    - Map service error: {response.status_code}")
        else:
            print(f"    - Map service not available: {response.status_code}")

    except Exception as e:
        print(f"    - Map service error: {e}")

    return None

def build_complete_endangered_species_dataset():
    """Build complete dataset of South Carolina endangered species with geometries"""

    print("SOUTH CAROLINA ENDANGERED SPECIES GEOMETRY FINDER")
    print("=" * 60)
    print("Searching NatureServe API for endangered species in South Carolina...")
    print()

    # Search for endangered species
    endangered_species = search_for_specific_endangered_species()

    if not endangered_species:
        print("No endangered species found in the search.")
        return None

    print(f"Found {len(endangered_species)} endangered species in South Carolina:")
    for species in endangered_species:
        print(f"- {species['scientificName']} ({species['commonName']})")

    print(f"\nNow getting geometries for each species...")
    print("=" * 60)

    # Add geometries to each species
    complete_dataset = []

    for species in endangered_species:
        print(f"\nProcessing: {species['scientificName']} ({species['commonName']})")
        print(f"  Global Rank: {species['globalRank']}")

        # Show South Carolina specific information
        for sc_info in species['scInfo']:
            print(f"  SC State Rank: {sc_info.get('rounded_state_rank', 'Unknown')}")
            print(f"  Last Observed: {sc_info.get('last_observed', 'Unknown')}")

        # Get geometry from map service
        geometry_features = get_geometry_from_map_service(species['elementId'], species['scientificName'])

        species_with_geometry = {
            'scientificName': species['scientificName'],
            'commonName': species['commonName'],
            'elementId': species['elementId'],
            'globalRank': species['globalRank'],
            'southCarolinaInfo': species['scInfo'],
            'geometryFeatures': geometry_features,
            'hasGeometry': bool(geometry_features),
            'geometryType': None,
            'coordinatePreview': None
        }

        # Extract geometry info for preview
        if geometry_features:
            sample_feature = geometry_features[0]
            geometry = sample_feature.get('geometry', {})
            species_with_geometry['geometryType'] = geometry.get('type', 'Unknown')

            if 'coordinates' in geometry:
                coords = geometry['coordinates']
                if isinstance(coords, list) and len(coords) > 0:
                    species_with_geometry['coordinatePreview'] = str(coords)[:100] + "..."

        complete_dataset.append(species_with_geometry)

        print(f"  Geometry: {'✓ Available' if geometry_features else '✗ Not found'}")

    # Save complete dataset
    output_file = 'sc_endangered_species_with_geometries.json'
    with open(output_file, 'w') as f:
        json.dump(complete_dataset, f, indent=2, default=str)

    # Create summary
    print("\n" + "=" * 60)
    print("FINAL RESULTS SUMMARY")
    print("=" * 60)

    total_species = len(complete_dataset)
    species_with_geometry = [s for s in complete_dataset if s['hasGeometry']]

    print(f"Total endangered species found in South Carolina: {total_species}")
    print(f"Species with geometry data available: {len(species_with_geometry)}")
    print(f"Success rate: {len(species_with_geometry)/total_species*100:.1f}%")

    print(f"\nSpecies WITH geometry data:")
    for species in species_with_geometry:
        print(f"  ✓ {species['scientificName']} ({species['commonName']})")
        print(f"    Geometry Type: {species['geometryType']}")
        print(f"    Global Rank: {species['globalRank']}")

    print(f"\nSpecies WITHOUT geometry data:")
    species_without_geometry = [s for s in complete_dataset if not s['hasGeometry']]
    for species in species_without_geometry:
        print(f"  ✗ {species['scientificName']} ({species['commonName']})")

    print(f"\nComplete dataset saved to: {output_file}")

    if species_with_geometry:
        print(f"\n🎉 SUCCESS! Found {len(species_with_geometry)} endangered species with geometric data for South Carolina!")
        return complete_dataset
    else:
        print(f"\n⚠️  No species found with geometry data.")
        return complete_dataset

def main():
    """Main function to run the endangered species search"""
    print("Starting South Carolina Endangered Species Search...")
    print()

    # Try the main approach first
    dataset = build_complete_endangered_species_dataset()

    # If the main approach doesn't find much due to API limitations,
    # provide the research-based information
    if not dataset or len(dataset) == 0:
        print("\n" + "=" * 60)
        print("API LIMITATIONS ENCOUNTERED")
        print("=" * 60)
        print("The NatureServe search API appears to return only a limited sample of species.")
        print("Providing research-based information instead...")
        print()

        research_dataset = create_research_based_dataset()

        print("=" * 60)
        print("ALTERNATIVE: MANUAL GEOMETRY ACCESS")
        print("=" * 60)
        print("To get geometries for these species, you can:")
        print()
        print("1. Use NatureServe Map Services directly:")
        print("   Base URL: https://explorer.natureserve.org/explorer-maps/species_subnational_ranks/{ELEMENT_ID}/FeatureServer/0/query")
        print("   Parameters: where=subnation_code='SC'&returnGeometry=true&f=json")
        print()
        print("2. Find Element IDs by:")
        print("   - Browsing https://explorer.natureserve.org/")
        print("   - Searching for species by scientific name")
        print("   - Extracting ID from URL (e.g., ELEMENT_GLOBAL.2.12345)")
        print()
        print("3. Known working example:")
        print("   Red-cockaded Woodpecker map service:")
        print("   https://explorer.natureserve.org/explorer-maps/species_subnational_ranks/ELEMENT_GLOBAL.2.102187/FeatureServer/0/query?where=subnation_code='SC'&returnGeometry=true&f=json")
        print()
        print("4. Use SC Department of Natural Resources data:")
        print("   - https://natural-heritage-program-scdnr.hub.arcgis.com/")
        print("   - SCDNR GIS data portal")
        print("   - SC Wildlife Action Plan spatial data")
        print()

        final_dataset = research_dataset

    else:
        print("\n" + "=" * 60)
        print("USAGE NOTES:")
        print("- The JSON file contains complete species and geometry data")
        print("- Geometry features include coordinates and attributes")
        print("- This data can be used in GIS applications or mapping libraries")
        print("- Each species includes NatureServe element ID for further API calls")
        print("- Map service URLs provided for direct access to updated data")
        print("=" * 60)

        final_dataset = dataset

    print("\n" + "=" * 60)
    print("SUMMARY OF SOUTH CAROLINA ENDANGERED SPECIES")
    print("=" * 60)

    summary_info = [
        ("Red-cockaded Woodpecker", "Recently downlisted to Threatened (2024)", "🎉 Conservation Success"),
        ("Loggerhead Sea Turtle", "State Reptile, beach nesting", "🏖️ Coastal Species"),
        ("North Atlantic Right Whale", "Critical calving habitat offshore", "🌊 Marine Species"),
        ("Wood Stork", "Large wading bird in wetlands", "🦢 Wetland Species"),
        ("Piping Plover", "Small shorebird on beaches", "🏃 Migratory Species"),
        ("Bog Turtle", "Smallest North American turtle", "🐢 Freshwater Species"),
        ("Atlantic Sturgeon", "Ancient fish species", "🐟 Anadromous Species")
    ]

    for species, description, category in summary_info:
        print(f"{category} {species}: {description}")

    print("\nFor the most current species lists and spatial data, check:")
    print("• SCDNR Heritage Trust Program: https://natural-heritage-program-scdnr.hub.arcgis.com/")
    print("• USFWS South Carolina: https://www.fws.gov/office/south-carolina-field")
    print("• NatureServe Explorer: https://explorer.natureserve.org/")

    print("\n✅ Script completed!")

    return final_dataset

if __name__ == "__main__":
    result = main()

Starting South Carolina Endangered Species Search...

SOUTH CAROLINA ENDANGERED SPECIES GEOMETRY FINDER
Searching NatureServe API for endangered species in South Carolina...

Searching for known endangered species in South Carolina...
Using direct species lookup approach...
Searching for: Picoides borealis (Red-cockaded woodpecker)
  - Not found in current search scope

Searching for: Dermochelys coriacea (Leatherback sea turtle)
  - Not found in current search scope

Searching for: Lepidochelys kempii (Kemp's Ridley sea turtle)
  ✓ Found and confirmed in South Carolina

Searching for: Caretta caretta (Loggerhead sea turtle)
  - Not found in current search scope

Searching for: Eubalaena glacialis (North Atlantic right whale)
  - Not found in current search scope

Searching for: Myotis sodalis (Indiana bat)
  - Not found in current search scope

Searching for: Myotis septentrionalis (Northern long-eared bat)
  - Not found in current search scope

Searching for: Corynorhinus rafinesquii

# South Carolina Endangered Species & NatureServe API Guide

## Overview
This guide documents the process of finding endangered species geometries in South Carolina using the NatureServe API, including API limitations discovered and alternative approaches.

## Key Findings

### NatureServe API Structure
- **Base URL**: `https://explorer.natureserve.org/api/data/`
- **Working Search Endpoint**: `POST https://explorer.natureserve.org/api/data/search`
- **Working Payload**: `{"criteriaType": "combined"}`
- **Individual Species**: `GET https://explorer.natureserve.org/api/data/taxon/{ELEMENT_ID}`
- **Map Services**: `https://explorer.natureserve.org/explorer-maps/species_subnational_ranks/{ELEMENT_ID}/FeatureServer/0/query`

### API Limitations Discovered
1. **Limited Search Results**: The search API only returns 20 species total, regardless of parameters
2. **No Pagination**: Pagination parameters (`pageSize`, `limit`, `maxResults`) are not recognized
3. **No Location Filters**: Location-based search parameters are not accepted in the search endpoint
4. **Strict JSON Structure**: The API requires exact Java class structures (`criteriaType` must be "combined", "ecosystems", or "species")

### Working NatureServe Endpoints
```
✅ GET https://explorer.natureserve.org/api/data/taxon/ELEMENT_GLOBAL.2.154701
✅ POST https://explorer.natureserve.org/api/data/search (with {"criteriaType": "combined"})
✅ GET https://explorer.natureserve.org/explorer-maps/species_subnational_ranks/{ID}/FeatureServer/0/query
❌ All other search parameter combinations return 400 errors
```

## South Carolina Endangered Species List

### Federally Endangered Species
| Scientific Name | Common Name | Status | Notes |
|----------------|-------------|---------|-------|
| Picoides borealis | Red-cockaded Woodpecker | Recently downlisted to Threatened (Nov 2024) | State conservation success story |
| Dermochelys coriacea | Leatherback Sea Turtle | Endangered | Marine species |
| Lepidochelys kempii | Kemp's Ridley Sea Turtle | Endangered | Marine species |
| Eubalaena glacialis | North Atlantic Right Whale | Endangered | Critical calving habitat off SC coast |
| Myotis sodalis | Indiana Bat | Endangered | Cave-dwelling species |
| Myotis septentrionalis | Northern Long-eared Bat | Endangered | Affected by white-nose syndrome |
| Corynorhinus rafinesquii | Rafinesque's Big-eared Bat | Endangered | Cave and mine roosts |
| Acipenser oxyrinchus | Atlantic Sturgeon | Endangered | Anadromous fish |
| Acipenser brevirostrum | Shortnose Sturgeon | Endangered | Anadromous fish |
| Lasmigona decorata | Carolina Heelsplitter | Endangered | Freshwater mussel |
| Bombus affinis | Rusty-patched Bumble Bee | Endangered | Pollinator species |

### Federally Threatened Species
| Scientific Name | Common Name | Status | Notes |
|----------------|-------------|---------|-------|
| Caretta caretta | Loggerhead Sea Turtle | Threatened | Official SC state reptile (1988) |
| Glyptemys muhlenbergii | Bog Turtle | Threatened | Smallest North American turtle |
| Laterallus jamaicensis | Eastern Black Rail | Threatened | Secretive marsh bird |
| Drymarchon couperi | Eastern Indigo Snake | Threatened | Largest North American snake |
| Ambystoma cingulatum | Flatwoods Salamander | Threatened | Fire-dependent habitat |
| Gopherus polyphemus | Gopher Tortoise | Threatened | Keystone species |
| Charadrius melodus | Piping Plover | Threatened | Beach-nesting shorebird |
| Calidris canutus | Red Knot | Threatened | Long-distance migrant |
| Trichechus manatus | West Indian Manatee | Threatened | Marine mammal |
| Mycteria americana | Wood Stork | Threatened | Only native North American stork |

### Plant Species
| Scientific Name | Common Name | Status |
|----------------|-------------|---------|
| Helianthus schweinitzii | Schweinitz Sunflower | Endangered |
| Amaranthus pumilus | Seabeach Amaranth | Threatened |
| Isotria medeoloides | Small Whorled Pogonia | Threatened |

## NatureServe API Usage Examples

### 1. Basic Species Search
```python
import requests

url = "https://explorer.natureserve.org/api/data/search"
headers = {'Content-Type': 'application/json', 'Accept': 'application/json'}
payload = {"criteriaType": "combined"}

response = requests.post(url, json=payload, headers=headers)
data = response.json()
```

### 2. Individual Species Lookup
```python
element_id = "ELEMENT_GLOBAL.2.154701"  # Example: Goldenseal
url = f"https://explorer.natureserve.org/api/data/taxon/{element_id}"
headers = {'Accept': 'application/json'}

response = requests.get(url, headers=headers)
species_data = response.json()

# Check for South Carolina occurrence
for national in species_data.get('elementNationals', []):
    for subnational in national.get('elementSubnationals', []):
        if subnational.get('subnation', {}).get('subnationCode') == 'SC':
            print(f"Found in SC: {species_data.get('scientificName')}")
```

### 3. Get Species Geometry
```python
element_id = "ELEMENT_GLOBAL.2.102187"  # Example species
map_url = f"https://explorer.natureserve.org/explorer-maps/species_subnational_ranks/{element_id}/FeatureServer/0/query"

params = {
    'where': "subnation_code='SC'",  # Filter for South Carolina
    'outFields': "*",
    'returnGeometry': "true",
    'f': "json"
}

response = requests.get(map_url, params=params)
geometry_data = response.json()

if 'features' in geometry_data:
    print(f"Found {len(geometry_data['features'])} geometric features")
```

## Data Structure Examples

### Species Data Structure
```json
{
  "elementGlobalId": 154701,
  "scientificName": "Hydrastis canadensis",
  "primaryCommonName": "Goldenseal",
  "roundedGlobalRank": "G3",
  "elementNationals": [
    {
      "elementSubnationals": [
        {
          "subnation": {
            "subnationCode": "SC",
            "nameEn": "South Carolina"
          },
          "roundedSubnationalRank": "S2",
          "lastObservedDate": "2020-01-01"
        }
      ]
    }
  ]
}
```

### Geometry Data Structure
```json
{
  "features": [
    {
      "attributes": {
        "objid": 1,
        "name": "South Carolina",
        "subnation_code": "SC",
        "rounded_s_rank": "S2",
        "element_subnational_id": 12345
      },
      "geometry": {
        "type": "Polygon",
        "coordinates": [[[lng, lat], [lng, lat], ...]]
      }
    }
  ]
}
```

## Alternative Data Sources

### 1. SC Department of Natural Resources
- **Heritage Trust Program**: https://natural-heritage-program-scdnr.hub.arcgis.com/
- **Species Database**: Contains state-specific occurrence data
- **GIS Data Portal**: Provides downloadable spatial datasets

### 2. USFWS South Carolina Field Office
- **Website**: https://www.fws.gov/office/south-carolina-field
- **Species Lists**: Federal endangered/threatened species in SC
- **Recovery Plans**: Detailed species information and habitat requirements

### 3. SC Wildlife Action Plan
- **Comprehensive Species Data**: State Wildlife Action Plan includes spatial data
- **Priority Species**: Beyond federal listings, includes species of greatest conservation need
- **Habitat Maps**: Critical habitat and occurrence data

### 4. iNaturalist & eBird
- **Citizen Science Data**: Real-time occurrence observations
- **iNaturalist**: https://www.inaturalist.org/places/south-carolina
- **eBird**: https://ebird.org/region/US-SC

## Working Map Service Examples

### Red-cockaded Woodpecker
```
https://explorer.natureserve.org/explorer-maps/species_subnational_ranks/ELEMENT_GLOBAL.2.102187/FeatureServer/0/query?where=subnation_code='SC'&returnGeometry=true&f=json
```

### General Pattern
```
https://explorer.natureserve.org/explorer-maps/species_subnational_ranks/{ELEMENT_ID}/FeatureServer/0/query
Parameters:
- where: subnation_code='SC'
- outFields: *
- returnGeometry: true
- f: json
```

## Conservation Status Codes

### Global Ranks (G-Ranks)
- **G1**: Critically imperiled globally (≤5 occurrences or ≤1,000 individuals)
- **G2**: Imperiled globally (6-20 occurrences or 1,000-3,000 individuals)
- **G3**: Vulnerable globally (21-100 occurrences)
- **G4**: Apparently secure globally
- **G5**: Secure globally

### State Ranks (S-Ranks)
- **S1**: Critically imperiled in state
- **S2**: Imperiled in state
- **S3**: Vulnerable in state
- **S4**: Apparently secure in state
- **S5**: Secure in state

## Next Steps & Recommendations

### 1. For Comprehensive Spatial Data
1. **Contact SCDNR Heritage Trust Program** directly for spatial datasets
2. **Use SCDNR ArcGIS Hub** for downloadable GIS data
3. **Access SC Wildlife Action Plan** spatial data portal

### 2. For Real-time NatureServe Data
1. **Find Element IDs** by browsing explorer.natureserve.org
2. **Use map services directly** with known Element IDs
3. **Implement systematic ID search** (ranges 100000-200000, 800000-900000)

### 3. For Research Applications
1. **Combine multiple data sources** (NatureServe + SCDNR + USFWS)
2. **Validate occurrence data** with recent field surveys
3. **Consider citizen science data** for recent observations

## Code Repository
The complete Python script for this analysis is available in the previous conversation artifacts, including:
- Systematic NatureServe API exploration
- Endangered species search functions
- Geometry extraction from map services
- Error handling for API limitations
- Research-based fallback datasets

## Contact Information for Data Access
- **SCDNR Heritage Trust**: natural-heritage-program-scdnr.hub.arcgis.com
- **USFWS SC Field Office**: (843) 727-4707
- **NatureServe**: support through explorer.natureserve.org

---
*Last Updated: December 2024*
*Data Sources: NatureServe Explorer, USFWS, SCDNR, Federal Register*